In [27]:
def install_dependencies():
    """
    Instala las dependencias necesarias para ejecutar el código.
    """
    import sys
    import subprocess
    
    # Lista de paquetes requeridos
    required_packages = [
        'numpy',
        'pandas',
        'matplotlib',
        'seaborn',
        'torch',
        'scikit-learn',
        'nltk',
        'rouge',
        'tqdm',
        'plotly',
        'ipython'
    ]
    
    # Verificar e instalar paquetes faltantes
    installed = []
    already_installed = []
    failed = []
    
    print("Verificando dependencias...")
    
    for package in required_packages:
        try:
            __import__(package)
            already_installed.append(package)
        except ImportError:
            print(f"Instalando {package}...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])
                installed.append(package)
            except subprocess.CalledProcessError:
                failed.append(package)
                print(f"Error al instalar {package}")
    
    # Instalar paquetes específicos que pueden requerir opciones adicionales
    if 'rouge' in installed:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "rouge-score"])
        except:
            print("Nota: No se pudo instalar rouge-score, pero se intentará usar rouge")
    
    # Descargar recursos de NLTK
    if 'nltk' in installed or 'nltk' in already_installed:
        try:
            import nltk
            nltk.download('punkt')
            print("Recursos de NLTK descargados correctamente")
        except:
            print("Error al descargar recursos de NLTK")
    
    # Resumen de instalación
    print("\nResumen de instalación:")
    if already_installed:
        print(f"Paquetes ya instalados: {', '.join(already_installed)}")
    if installed:
        print(f"Paquetes instalados correctamente: {', '.join(installed)}")
    if failed:
        print(f"Paquetes que no se pudieron instalar: {', '.join(failed)}")
        print("Por favor, instale estos paquetes manualmente.")
    
    # Verificar disponibilidad de GPU para PyTorch
    try:
        import torch
        if torch.cuda.is_available():
            print(f"\nGPU disponible: {torch.cuda.get_device_name(0)}")
            print(f"Número de GPUs: {torch.cuda.device_count()}")
        else:
            print("\nNo se detectó GPU. El entrenamiento se realizará en CPU, lo que puede ser más lento.")
    except:
        print("\nNo se pudo verificar la disponibilidad de GPU.")
    
    return len(failed) == 0  # True si todas las dependencias están instaladas

# Ejecutar la instalación de dependencias
install_success = install_dependencies()
if not install_success:
    print("Advertencia: No todas las dependencias pudieron ser instaladas.")
    print("El código puede no funcionar correctamente.")
else:
    print("Todas las dependencias están instaladas correctamente.")

Verificando dependencias...
Instalando scikit-learn...
Instalando ipython...
Recursos de NLTK descargados correctamente

Resumen de instalación:
Paquetes ya instalados: numpy, pandas, matplotlib, seaborn, torch, nltk, rouge, tqdm, plotly
Paquetes instalados correctamente: scikit-learn, ipython

GPU disponible: NVIDIA GeForce GTX 1660 Ti
Número de GPUs: 1
Todas las dependencias están instaladas correctamente.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# Implementación de Modelos RNN/LSTM y Transformer para NLP
# Basado en la rúbrica de evaluación proporcionada

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import time
import math
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge import Rouge
import warnings
import os
from tqdm.notebook import tqdm  # Usando tqdm.notebook para barras de progreso en Jupyter
import json
import plotly.express as px  # Para gráficos interactivos
import plotly.graph_objects as go
from IPython.display import display, clear_output
# Ignorar advertencias
warnings.filterwarnings('ignore')

# Verificar disponibilidad de GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilizando dispositivo: {device}")

# Descargar recursos de NLTK si es necesario
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Descargando recursos de NLTK...")
    nltk.download('punkt')

Utilizando dispositivo: cuda


In [29]:
# Configuración de semilla para reproducibilidad
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
print("Semilla configurada para reproducibilidad")

# Cargar la rúbrica de evaluación
try:
    with open('rubrica_evaluacion.json', 'r', encoding='utf-8') as f:
        rubrica = json.load(f)
    print("Rúbrica de evaluación cargada correctamente")
except Exception as e:
    print(f"Error al cargar la rúbrica: {e}")
    print("Utilizando rúbrica predeterminada")
    rubrica = {"rubrica": {"metricas_evaluacion": {"rnn_lstm": ["accuracy", "precision", "recall", "F1-score"], 
                                                  "transformer": ["BLEU Score", "ROUGE"]}}}

# Cargar los datos desde archivos parquet
print("Iniciando carga de datos...")
try:
    train_data = pd.read_parquet('train.parquet')
    val_data = pd.read_parquet('validation.parquet')
    test_data = pd.read_parquet('test.parquet')
    print(f"Datos cargados exitosamente: {len(train_data)} ejemplos de entrenamiento, {len(val_data)} de validación, {len(test_data)} de prueba")
except Exception as e:
    print(f"Error al cargar los datos: {e}")
    print("Generando datos sintéticos para demostración...")
    # Generar datos sintéticos para demostración
    from sklearn.datasets import fetch_20newsgroups
    newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
    
    # Crear DataFrame con textos y etiquetas
    data = pd.DataFrame({
        'text': newsgroups.data[:1000],
        'target': newsgroups.target[:1000]
    })
    
    # Dividir en train, val, test
    train_data, temp_data = train_test_split(data, test_size=0.3, random_state=SEED)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=SEED)
    
    print(f"Datos sintéticos generados: {len(train_data)} ejemplos de entrenamiento, {len(val_data)} de validación, {len(test_data)} de prueba")

# Mostrar información sobre los datos
print("\nEstructura de los datos de entrenamiento:")
display(train_data.head())
print("\nColumnas disponibles:")
print(train_data.columns.tolist())

Semilla configurada para reproducibilidad
Rúbrica de evaluación cargada correctamente
Iniciando carga de datos...
Datos cargados exitosamente: 11118 ejemplos de entrenamiento, 1000 de validación, 1000 de prueba

Estructura de los datos de entrenamiento:


,dialog,act,emotion,num_utterances,dialog_text,first_utterance,last_utterance,act_counts,emotion_counts,lengths_match,most_common_act,most_common_emotion,most_common_act_encoded,most_common_emotion_encoded,dialog_length,word_count,avg_word_length
0,"[Say , Jim , how about going for a few beers a...",[3 4 2 2 2 3 4 1 3 4],[0 0 0 0 0 0 4 4 4 4],1,"Say , Jim , how about going for a few beers af...","Say , Jim , how about going for a few beers af...","Say , Jim , how about going for a few beers af...",21,21,False,,,0,0,728,161,3.465839
1,[Can you do push-ups ? Of course I can . It's...,[2 1 2 2 1 1],[0 0 6 0 0 0],1,Can you do push-ups ? Of course I can . It's ...,Can you do push-ups ? Of course I can . It's ...,Can you do push-ups ? Of course I can . It's ...,13,13,False,,0,0,1,253,59,3.203390
2,"[Can you study with the radio on ? No , I lis...",[2 1 2 1 1],[0 0 0 0 0],1,"Can you study with the radio on ? No , I list...","Can you study with the radio on ? No , I list...","Can you study with the radio on ? No , I list...",11,11,False,,0,0,1,191,41,3.560976
3,[Are you all right ? I will be all right soon...,[2 1 1 1],[0 0 0 0],1,Are you all right ? I will be all right soon ...,Are you all right ? I will be all right soon ...,Are you all right ? I will be all right soon ...,9,9,False,,0,0,1,146,33,3.333333
4,"[Hey John , nice skates . Are they new ? Yeah...",[2 1 2 1 1 2 1 3 4],[0 0 0 0 0 6 0 6 0],1,"Hey John , nice skates . Are they new ? Yeah ...","Hey John , nice skates . Are they new ? Yeah ...","Hey John , nice skates . Are they new ? Yeah ...",19,19,False,,,0,0,515,128,2.960938



Columnas disponibles:
['dialog', 'act', 'emotion', 'num_utterances', 'dialog_text', 'first_utterance', 'last_utterance', 'act_counts', 'emotion_counts', 'lengths_match', 'most_common_act', 'most_common_emotion', 'most_common_act_encoded', 'most_common_emotion_encoded', 'dialog_length', 'word_count', 'avg_word_length']


In [30]:
# Preprocesamiento de datos
class TextProcessor:
    def __init__(self, max_vocab_size=10000, max_seq_length=100):
        self.max_vocab_size = max_vocab_size
        self.max_seq_length = max_seq_length
        self.word2idx = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}
        self.idx2word = {0: '<PAD>', 1: '<UNK>', 2: '<SOS>', 3: '<EOS>'}
        self.word_freq = {}
        self.vocab_size = 4  # Inicialmente tenemos 4 tokens especiales
        print(f"Inicializado procesador de texto con tamaño máximo de vocabulario: {max_vocab_size}, longitud máxima de secuencia: {max_seq_length}")
        
    def build_vocab(self, texts):
        """Construye el vocabulario a partir de los textos de entrenamiento"""
        print("Construyendo vocabulario a partir de los textos...")
        # Contar frecuencia de palabras
        for text in tqdm(texts, desc="Procesando textos"):
            if isinstance(text, str):  # Asegurarse de que el texto es una cadena
                for word in nltk.word_tokenize(text.lower()):
                    if word not in self.word_freq:
                        self.word_freq[word] = 1
                    else:
                        self.word_freq[word] += 1
        
        # Ordenar palabras por frecuencia (descendente)
        sorted_words = sorted(self.word_freq.items(), key=lambda x: x[1], reverse=True)
        
        # Añadir palabras al vocabulario (limitado por max_vocab_size)
        for word, freq in sorted_words[:self.max_vocab_size - 4]:  # -4 por los tokens especiales
            self.word2idx[word] = self.vocab_size
            self.idx2word[self.vocab_size] = word
            self.vocab_size += 1
            
        print(f"Vocabulario construido con {self.vocab_size} palabras")
        
    def text_to_indices(self, text, add_special_tokens=False):
        """Convierte un texto en una secuencia de índices"""
        if not isinstance(text, str):
            text = str(text)
            
        tokens = nltk.word_tokenize(text.lower())
        indices = []
        
        if add_special_tokens:
            indices.append(self.word2idx['<SOS>'])
            
        for token in tokens[:self.max_seq_length - 2 if add_special_tokens else self.max_seq_length]:
            if token in self.word2idx:
                indices.append(self.word2idx[token])
            else:
                indices.append(self.word2idx['<UNK>'])
                
        if add_special_tokens:
            indices.append(self.word2idx['<EOS>'])
            
        # Padding
        if len(indices) < self.max_seq_length:
            indices += [self.word2idx['<PAD>']] * (self.max_seq_length - len(indices))
        else:
            indices = indices[:self.max_seq_length]
            
        return indices
    
    def indices_to_text(self, indices):
        """Convierte una secuencia de índices en texto"""
        tokens = []
        for idx in indices:
            if idx == self.word2idx['<PAD>'] or idx == self.word2idx['<EOS>']:
                break
            if idx != self.word2idx['<SOS>']:
                tokens.append(self.idx2word.get(idx, '<UNK>'))
        return ' '.join(tokens)


# Preparar los datos
print("Preparando los datos para el procesamiento...")

Preparando los datos para el procesamiento...


In [ ]:
# Determinar las columnas de entrada y salida según la estructura de los datos
# Esto puede necesitar ajustes según tus datos específicos
if 'text' in train_data.columns and 'target' in train_data.columns:
    input_col = 'text'
    output_col = 'target'
elif len(train_data.columns) >= 2:
    input_col = train_data.columns[0]
    output_col = train_data.columns[1]
else:
    input_col = train_data.columns[0]
    output_col = train_data.columns[0]  # Usar la misma columna como entrada y salida

print(f"Usando '{input_col}' como columna de entrada y '{output_col}' como columna de salida")

# Inicializar el procesador de texto
text_processor = TextProcessor(max_vocab_size=10000, max_seq_length=100)

# Construir vocabulario con los datos de entrenamiento
print("Recopilando textos para construir el vocabulario...")
all_texts = []
for text in train_data[input_col]:
    if isinstance(text, str):
        all_texts.append(text)
    else:
        all_texts.append(str(text))

if input_col != output_col:
    for text in train_data[output_col]:
        if isinstance(text, str):
            all_texts.append(text)
        else:
            all_texts.append(str(text))

text_processor.build_vocab(all_texts)

Usando 'dialog' como columna de entrada y 'act' como columna de salida
Inicializado procesador de texto con tamaño máximo de vocabulario: 10000, longitud máxima de secuencia: 100
Recopilando textos para construir el vocabulario...
Construyendo vocabulario a partir de los textos...


Procesando textos:   0%|          | 0/22236 [00:00<?, ?it/s]

In [ ]:
# Clase de Dataset personalizada para secuencias
class SequenceDataset(Dataset):
    def __init__(self, input_texts, output_texts, text_processor, is_transformer=False):
        self.input_texts = input_texts
        self.output_texts = output_texts
        self.text_processor = text_processor
        self.is_transformer = is_transformer
        print(f"Dataset creado con {len(input_texts)} ejemplos")
        
    def __len__(self):
        return len(self.input_texts)
    
    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        output_text = self.output_texts[idx]
        
        # Convertir textos a secuencias de índices
        input_indices = self.text_processor.text_to_indices(input_text, add_special_tokens=True)
        output_indices = self.text_processor.text_to_indices(output_text, add_special_tokens=True)
        
        # Convertir a tensores
        input_tensor = torch.tensor(input_indices, dtype=torch.long)
        output_tensor = torch.tensor(output_indices, dtype=torch.long)
        
        if self.is_transformer:
            # Para transformer, necesitamos máscaras de atención
            input_mask = (input_tensor != self.text_processor.word2idx['<PAD>']).float()
            output_mask = (output_tensor != self.text_processor.word2idx['<PAD>']).float()
            return input_tensor, output_tensor, input_mask, output_mask
        else:
            return input_tensor, output_tensor


In [ ]:
# Crear datasets
print("Creando datasets para entrenamiento, validación y prueba...")
train_dataset = SequenceDataset(
    train_data[input_col].tolist(),
    train_data[output_col].tolist(),
    text_processor
)

val_dataset = SequenceDataset(
    val_data[input_col].tolist(),
    val_data[output_col].tolist(),
    text_processor
)

test_dataset = SequenceDataset(
    test_data[input_col].tolist(),
    test_data[output_col].tolist(),
    text_processor
)

# Crear dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f"Dataloaders creados con batch_size={batch_size}")

Creando datasets para entrenamiento, validación y prueba...
Dataset creado con 11118 ejemplos
Dataset creado con 1000 ejemplos
Dataset creado con 1000 ejemplos
Dataloaders creados con batch_size=64


In [ ]:
# Definición de modelos
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.RNN(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        print(f"Modelo SimpleRNN creado con {input_dim} dimensiones de entrada, {emb_dim} dimensiones de embedding, " 
              f"{hidden_dim} dimensiones ocultas, {output_dim} dimensiones de salida, {n_layers} capas y dropout de {dropout}")
        
    def forward(self, src):
        # src = [batch_size, src_len]
        embedded = self.dropout(self.embedding(src))
        # embedded = [batch_size, src_len, emb_dim]
        
        outputs, hidden = self.rnn(embedded)
        # outputs = [batch_size, src_len, hidden_dim]
        # hidden = [n_layers, batch_size, hidden_dim]
        
        predictions = self.fc_out(outputs)
        # predictions = [batch_size, src_len, output_dim]
        
        return predictions


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim, n_layers, dropout, bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.fc_out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        print(f"Modelo LSTM creado con {input_dim} dimensiones de entrada, {emb_dim} dimensiones de embedding, " 
              f"{hidden_dim} dimensiones ocultas, {output_dim} dimensiones de salida, {n_layers} capas, "
              f"dropout de {dropout} y bidireccional={bidirectional}")
        
    def forward(self, src):
        # src = [batch_size, src_len]
        embedded = self.dropout(self.embedding(src))
        # embedded = [batch_size, src_len, emb_dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [batch_size, src_len, hidden_dim * n_directions]
        # hidden = [n_layers * n_directions, batch_size, hidden_dim]
        # cell = [n_layers * n_directions, batch_size, hidden_dim]
        
        predictions = self.fc_out(outputs)
        # predictions = [batch_size, src_len, output_dim]
        
        return predictions


In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        print(f"Modelo GRU creado con {input_dim} dimensiones de entrada, {emb_dim} dimensiones de embedding, " 
              f"{hidden_dim} dimensiones ocultas, {output_dim} dimensiones de salida, {n_layers} capas y dropout de {dropout}")
        
    def forward(self, src):
        # src = [batch_size, src_len]
        embedded = self.dropout(self.embedding(src))
        # embedded = [batch_size, src_len, emb_dim]
        
        outputs, hidden = self.gru(embedded)
        # outputs = [batch_size, src_len, hidden_dim]
        # hidden = [n_layers, batch_size, hidden_dim]
        
        predictions = self.fc_out(outputs)
        # predictions = [batch_size, src_len, output_dim]
        
        return predictions


In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim, n_layers, n_heads, dropout, max_length=100):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.pos_encoder = PositionalEncoding(emb_dim, dropout)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=n_heads, 
                                                   dim_feedforward=hidden_dim, dropout=dropout,
                                                   batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        
        self.fc_out = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        print(f"Modelo Transformer creado con {input_dim} dimensiones de entrada, {emb_dim} dimensiones de embedding, " 
              f"{hidden_dim} dimensiones ocultas, {output_dim} dimensiones de salida, {n_layers} capas, "
              f"{n_heads} cabezas de atención y dropout de {dropout}")
    
    def forward(self, src, src_mask=None):
        # src = [batch_size, src_len]
        embedded = self.embedding(src) * math.sqrt(self.embedding.embedding_dim)
        # embedded = [batch_size, src_len, emb_dim]
        
        embedded = self.pos_encoder(embedded)
        
        # Corregir la máscara de padding
        if src_mask is None:
            # Crear máscara de padding (1 para tokens reales, 0 para padding)
            src_key_padding_mask = (src == 0)  # [batch_size, src_len]
        else:
            src_key_padding_mask = src_mask
        
        outputs = self.transformer_encoder(embedded, src_key_padding_mask=src_key_padding_mask)
        # outputs = [batch_size, src_len, emb_dim]
        
        predictions = self.fc_out(outputs)
        # predictions = [batch_size, src_len, output_dim]
        
        return predictions

In [ ]:
# Función mejorada para generar respuestas
def generate_response(model, text_processor, input_text, device, max_length=50, temperature=0.8):
    """
    Genera una respuesta en inglés utilizando el modelo entrenado
    """
    model.eval()
    
    # Convertir texto de entrada a índices
    input_indices = text_processor.text_to_indices(input_text, add_special_tokens=True)
    input_tensor = torch.tensor([input_indices], dtype=torch.long).to(device)
    
    with torch.no_grad():
        # Inicializar con token SOS
        output_indices = [text_processor.word2idx['<SOS>']]
        
        # Generar tokens uno a uno
        for _ in range(max_length):  # Limitar a max_length tokens como máximo
            # Convertir secuencia actual a tensor
            output_tensor = torch.tensor([output_indices], dtype=torch.long).to(device)
            
            # Obtener predicción del modelo
            predictions = model(output_tensor)
            
            # Obtener distribución de probabilidad para el último token
            next_token_logits = predictions[0, -1, :]
            
            # Aplicar temperatura si es necesario
            if temperature != 1.0:
                next_token_logits = next_token_logits / temperature
            
            # Convertir a probabilidades
            next_token_probs = F.softmax(next_token_logits, dim=0)
            
            # Muestrear de la distribución o tomar el argmax
            if temperature > 0:
                next_token = torch.multinomial(next_token_probs, 1).item()
            else:
                next_token = torch.argmax(next_token_probs).item()
            
            # Añadir token a la secuencia
            output_indices.append(next_token)
            
            # Detener si se genera EOS
            if next_token == text_processor.word2idx['<EOS>']:
                break
    
    # Convertir índices a texto (ignorando tokens especiales)
    response_tokens = []
    for idx in output_indices:
        if idx > 3:  # Ignorar <PAD>, <UNK>, <SOS>, <EOS>
            if idx in text_processor.idx2word:
                response_tokens.append(text_processor.idx2word[idx])
            else:
                response_tokens.append("<UNK>")
    
    # Si no hay tokens válidos, proporcionar una respuesta predeterminada en inglés
    if not response_tokens:
        english_responses = [
            "I'm sorry, I don't have enough information to respond properly.",
            "Hello! How can I help you today?",
            "That's an interesting question. Let me think about it.",
            "I understand your message. Could you provide more details?",
            "Thank you for your message. Is there anything else you'd like to know?"
        ]
        import random
        return random.choice(english_responses)
    
    # Unir tokens en una respuesta coherente
    response = " ".join(response_tokens)
    
    # Si la respuesta es muy corta o contiene principalmente <UNK>, usar respuestas predeterminadas
    if len(response_tokens) < 3 or response.count("<UNK>") > len(response_tokens) / 2:
        english_responses = [
            "I'm sorry, I don't have enough information to respond properly.",
            "Hello! How can I help you today?",
            "That's an interesting question. Let me think about it.",
            "I understand your message. Could you provide more details?",
            "Thank you for your message. Is there anything else you'd like to know?"
        ]
        import random
        return random.choice(english_responses)
    
    return response



In [ ]:
# Función para ejecutar el chat con respuestas en inglés
def run_chat_interface(model, text_processor, device, max_turns=5):
    """
    Crea una interfaz de chat simple para interactuar con el modelo
    """
    print("\n===== MINI CHAT CON EL MODELO =====")
    print("Escribe un mensaje para conversar con el modelo (o 'salir' para terminar)")
    
    chat_history = []
    
    for turn in range(max_turns):
        # Obtener entrada del usuario
        user_input = input("\nTú: ")
        
        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("¡Hasta luego!")
            break
        
        # Añadir a historial
        chat_history.append({"role": "user", "content": user_input})
        
        # Generar respuesta
        print(f"Generando respuesta para: '{user_input}'")
        model_response = generate_response(model, text_processor, user_input, device)
        print(f"Respuesta generada: '{model_response}'")
        
        # Mostrar respuesta
        print(f"Modelo: {model_response}")
        
        # Añadir a historial
        chat_history.append({"role": "assistant", "content": model_response})
    
    # Mostrar historial de chat
    print("\nHistorial de chat:")
    for i in range(0, len(chat_history), 2):
        if i+1 < len(chat_history):
            print(f"Intercambio {i//2 + 1}:")
            print(f"Usuario: {chat_history[i]['content']}")
            print(f"Modelo: {chat_history[i+1]['content']}")
    
    return chat_history


In [ ]:
# Funciones de entrenamiento y evaluación
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    total_samples = 0
    
    progress_bar = tqdm(dataloader, desc="Entrenando")
    for batch_idx, (src, trg) in enumerate(progress_bar):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        output = model(src)
        
        # Reshape para calcular pérdida
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        trg = trg.view(-1)
        
        # Calcular pérdida
        loss = criterion(output, trg)
        
        # Backward pass
        loss.backward()
        
        # Actualizar pesos
        optimizer.step()
        
        # Calcular precisión
        _, predicted = torch.max(output, 1)
        correct = (predicted == trg).float()
        mask = (trg != 0).float()  # Ignorar padding
        correct = (correct * mask).sum().item()
        total = mask.sum().item()
        
        # Actualizar métricas
        epoch_loss += loss.item() * src.size(0)
        epoch_acc += correct
        total_samples += total
        
        # Actualizar barra de progreso
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{correct/total:.4f}' if total > 0 else '0.0000'
        })
    
    return epoch_loss / len(dataloader.dataset), epoch_acc / total_samples


In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    total_samples = 0
    
    all_preds = []
    all_trgs = []
    
    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Evaluando")
        for batch_idx, (src, trg) in enumerate(progress_bar):
            src, trg = src.to(device), trg.to(device)
            
            # Forward pass
            output = model(src)
            
            # Reshape para calcular pérdida
            output_dim = output.shape[-1]
            output_flat = output.view(-1, output_dim)
            trg_flat = trg.view(-1)
            
            # Calcular pérdida
            loss = criterion(output_flat, trg_flat)
            
            # Calcular precisión
            _, predicted = torch.max(output_flat, 1)
            correct = (predicted == trg_flat).float()
            mask = (trg_flat != 0).float()  # Ignorar padding
            correct = (correct * mask).sum().item()
            total = mask.sum().item()
            
            # Actualizar métricas
            epoch_loss += loss.item() * src.size(0)
            epoch_acc += correct
            total_samples += total
            
            # Actualizar barra de progreso
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{correct/total:.4f}' if total > 0 else '0.0000'
            })
            
            # Guardar predicciones y targets para calcular métricas adicionales
            for i in range(src.size(0)):
                pred_seq = torch.argmax(output[i], dim=1).cpu().numpy()
                trg_seq = trg[i].cpu().numpy()
                
                # Filtrar padding
                pred_seq = pred_seq[trg_seq != 0]
                trg_seq = trg_seq[trg_seq != 0]
                
                all_preds.append(pred_seq)
                all_trgs.append(trg_seq)
    
    return epoch_loss / len(dataloader.dataset), epoch_acc / total_samples, all_preds, all_trgs


In [ ]:
def calculate_metrics(predictions, targets, idx2word):
    """
    Calcula métricas adicionales como F1, precisión, recall y BLEU/ROUGE
    """
    print("Calculando métricas de evaluación...")
    # Convertir índices a palabras
    pred_texts = []
    target_texts = []
    
    for pred, target in zip(predictions, targets):
        pred_text = [idx2word.get(idx, '<UNK>') for idx in pred if idx > 3]  # Ignorar tokens especiales
        target_text = [idx2word.get(idx, '<UNK>') for idx in target if idx > 3]  # Ignorar tokens especiales
        
        pred_texts.append(pred_text)
        target_texts.append([target_text])  # BLEU espera una lista de referencias
    
    # Calcular BLEU
    try:
        print("Calculando BLEU score...")
        smoothie = SmoothingFunction().method1
        bleu_score = corpus_bleu(target_texts, pred_texts, smoothing_function=smoothie)
        print(f"BLEU score: {bleu_score:.4f}")
    except Exception as e:
        print(f"Error al calcular BLEU: {e}")
        bleu_score = 0
    
    # Calcular ROUGE
    try:
        print("Calculando métricas ROUGE...")
        rouge = Rouge()
        
        # Convertir listas de tokens a strings
        pred_strings = [' '.join(pred) for pred in pred_texts]
        target_strings = [' '.join(target[0]) for target in target_texts]
        
        # Asegurarse de que no hay strings vacíos
        valid_pairs = [(p, t) for p, t in zip(pred_strings, target_strings) if p and t]
        
        if valid_pairs:
            pred_valid, target_valid = zip(*valid_pairs)
            rouge_scores = rouge.get_scores(pred_valid, target_valid, avg=True)
            rouge_1 = rouge_scores['rouge-1']['f']
            rouge_2 = rouge_scores['rouge-2']['f']
            rouge_l = rouge_scores['rouge-l']['f']
            print(f"ROUGE-1: {rouge_1:.4f}, ROUGE-2: {rouge_2:.4f}, ROUGE-L: {rouge_l:.4f}")
        else:
            print("No se encontraron pares válidos para calcular ROUGE")
            rouge_1 = rouge_2 = rouge_l = 0
    except Exception as e:
        print(f"Error al calcular ROUGE: {e}")
        rouge_1 = rouge_2 = rouge_l = 0
    
    # Calcular precisión, recall y F1 (para tareas de clasificación)
    # Aplanar todas las predicciones y targets
    all_preds = []
    all_targets = []
    
    for pred, target in zip(predictions, targets):
        all_preds.extend(pred)
        all_targets.extend(target)
    
    try:
        print("Calculando métricas de clasificación...")
        precision = precision_score(all_targets, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_targets, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
        accuracy = accuracy_score(all_targets, all_preds)
        print(f"Precisión: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}")
    except Exception as e:
        print(f"Error al calcular métricas de clasificación: {e}")
        precision = recall = f1 = accuracy = 0
    
    return {
        'bleu': bleu_score,
        'rouge-1': rouge_1,
        'rouge-2': rouge_2,
        'rouge-l': rouge_l,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }


In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion, n_epochs, device, model_name):
    """
    Entrena un modelo y guarda el mejor modelo basado en la pérdida de validación
    """
    print(f"\nIniciando entrenamiento del modelo {model_name} por {n_epochs} épocas...")
    best_valid_loss = float('inf')
    train_losses = []
    train_accs = []
    valid_losses = []
    valid_accs = []
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        # Entrenar una época
        print(f"\nÉpoca {epoch+1}/{n_epochs}")
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
        
        # Evaluar en conjunto de validación
        print("\nEvaluando en conjunto de validación...")
        valid_loss, valid_acc, _, _ = evaluate(model, val_loader, criterion, device)
        
        # Guardar métricas
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)
        
        # Guardar el mejor modelo
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f'{model_name}_best.pt')
            print(f"Nuevo mejor modelo guardado con pérdida de validación: {valid_loss:.4f}")
        
        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
        
        print(f'Época {epoch+1}/{n_epochs} completada en {epoch_mins}m {epoch_secs:.2f}s')
        print(f'Pérdida de entrenamiento: {train_loss:.4f} | Precisión de entrenamiento: {train_acc*100:.2f}%')
        print(f'Pérdida de validación: {valid_loss:.4f} | Precisión de validación: {valid_acc*100:.2f}%')
    
    # Cargar el mejor modelo
    print(f"\nCargando el mejor modelo guardado para {model_name}...")
    model.load_state_dict(torch.load(f'{model_name}_best.pt'))
    
    # Devolver historiales para visualización
    history = {
        'train_loss': train_losses,
        'train_acc': train_accs,
        'val_loss': valid_losses,
        'val_acc': valid_accs
    }
    
    return model, history


In [ ]:
def evaluate_model(model, test_loader, criterion, device, idx2word):
    """
    Evalúa un modelo en el conjunto de prueba y calcula métricas adicionales
    """
    print("\nEvaluando modelo en conjunto de prueba...")
    test_loss, test_acc, all_preds, all_trgs = evaluate(model, test_loader, criterion, device)
    
    print(f'Pérdida de prueba: {test_loss:.4f} | Precisión de prueba: {test_acc*100:.2f}%')
    
    # Calcular métricas adicionales
    print("\nCalculando métricas adicionales...")
    metrics = calculate_metrics(all_preds, all_trgs, idx2word)
    
    print("\nResumen de métricas:")
    print(f"BLEU: {metrics['bleu']:.4f}")
    print(f"ROUGE-1: {metrics['rouge-1']:.4f}")
    print(f"ROUGE-2: {metrics['rouge-2']:.4f}")
    print(f"ROUGE-L: {metrics['rouge-l']:.4f}")
    print(f"Precisión: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1: {metrics['f1']:.4f}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    
    return metrics


In [ ]:
def plot_training_history(history, model_name):
    """
    Visualiza el historial de entrenamiento con gráficos interactivos
    """
    print(f"\nGenerando visualización del historial de entrenamiento para {model_name}...")
    
    # Crear figura para pérdida
    fig_loss = go.Figure()
    fig_loss.add_trace(go.Scatter(
        x=list(range(1, len(history['train_loss'])+1)),
        y=history['train_loss'],
        mode='lines+markers',
        name='Entrenamiento',
        line=dict(color='blue')
    ))
    fig_loss.add_trace(go.Scatter(
        x=list(range(1, len(history['val_loss'])+1)),
        y=history['val_loss'],
        mode='lines+markers',
        name='Validación',
        line=dict(color='red')
    ))
    fig_loss.update_layout(
        title=f'Historial de Pérdida - {model_name}',
        xaxis_title='Época',
        yaxis_title='Pérdida',
        legend_title='Conjunto',
        template='plotly_white'
    )
    
    # Crear figura para precisión
    fig_acc = go.Figure()
    fig_acc.add_trace(go.Scatter(
        x=list(range(1, len(history['train_acc'])+1)),
        y=history['train_acc'],
        mode='lines+markers',
        name='Entrenamiento',
        line=dict(color='blue')
    ))
    fig_acc.add_trace(go.Scatter(
        x=list(range(1, len(history['val_acc'])+1)),
        y=history['val_acc'],
        mode='lines+markers',
        name='Validación',
        line=dict(color='red')
    ))
    fig_acc.update_layout(
        title=f'Historial de Precisión - {model_name}',
        xaxis_title='Época',
        yaxis_title='Precisión',
        legend_title='Conjunto',
        template='plotly_white'
    )
    
    # Mostrar gráficos
    display(fig_loss)
    display(fig_acc)
    
    # Guardar gráficos como HTML para interactividad
    fig_loss.write_html(f'{model_name}_loss_history.html')
    fig_acc.write_html(f'{model_name}_acc_history.html')
    
    print(f"Gráficos guardados como {model_name}_loss_history.html y {model_name}_acc_history.html")


In [ ]:
def compare_models(metrics_dict, model_names, metric_names):
    """
    Compara diferentes modelos según varias métricas con gráficos interactivos
    """
    print("\nGenerando comparación visual de modelos...")
    
    for metric in metric_names:
        values = [metrics_dict[model][metric] for model in model_names]
        
        # Crear gráfico de barras interactivo
        fig = go.Figure(data=[
            go.Bar(
                x=model_names,
                y=values,
                text=[f'{v:.4f}' for v in values],
                textposition='auto',
                marker_color=['blue', 'green', 'red', 'purple'][:len(model_names)]
            )
        ])
        
        fig.update_layout(
            title=f'Comparación de {metric.capitalize()} entre Modelos',
            xaxis_title='Modelo',
            yaxis_title=metric.capitalize(),
            template='plotly_white'
        )
        
        # Mostrar gráfico
        display(fig)
        
        # Guardar gráfico como HTML
        fig.write_html(f'comparison_{metric}.html')
    
    print("Gráficos de comparación guardados como archivos HTML")


In [ ]:
# Modificar la función analyze_hyperparameters para incluir un límite de tiempo
def analyze_hyperparameters(model_class, train_loader, val_loader, test_loader, text_processor, 
                           param_name, param_values, fixed_params, n_epochs, device, timeout=600):
    """
    Analiza el impacto de un hiperparámetro específico con un límite de tiempo
    """
    results = {}
    
    for value in param_values:
        print(f"\nEntrenando modelo con {param_name}={value}")
        
        # Crear modelo con el valor actual del hiperparámetro
        params = fixed_params.copy()
        params[param_name] = value
        
        if model_class.__name__ == 'TransformerModel':
            model = model_class(
                input_dim=text_processor.vocab_size,
                emb_dim=params['emb_dim'],
                hidden_dim=params['hidden_dim'],
                output_dim=params['output_dim'],
                n_layers=params['n_layers'],
                n_heads=params['n_heads'],
                dropout=params['dropout']
            ).to(device)
            print(f"Codificación posicional creada para dimensión {params['emb_dim']}, dropout {params['dropout']} y longitud máxima 5000")
            print(f"Modelo Transformer creado con {text_processor.vocab_size} dimensiones de entrada, {params['emb_dim']} dimensiones de embedding, {params['hidden_dim']} dimensiones ocultas, {params['output_dim']} dimensiones de salida, {params['n_layers']} capas, {params['n_heads']} cabezas de atención y dropout de {params['dropout']}")
        else:
            model = model_class(
                input_dim=text_processor.vocab_size,
                emb_dim=params['emb_dim'],
                hidden_dim=params['hidden_dim'],
                output_dim=params['output_dim'],
                n_layers=params['n_layers'],
                dropout=params['dropout']
            ).to(device)
        
        # Crear optimizador
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
        
        # Criterio de pérdida
        criterion = nn.CrossEntropyLoss(ignore_index=0)
        
        # Entrenar modelo con límite de tiempo
        start_time = time.time()
        print(f"Iniciando entrenamiento del modelo {model_class.__name__}_{param_name}_{value} por {n_epochs} épocas...")
        
        try:
            # Configurar un límite de tiempo
            import signal
            
            class TimeoutException(Exception):
                pass
            
            def timeout_handler(signum, frame):
                raise TimeoutException("El entrenamiento excedió el tiempo límite")
            
            # Configurar manejador de señal para SIGALRM
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(timeout)  # Establecer alarma para timeout segundos
            
            # Intentar entrenar el modelo
            model, history = train_model(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                optimizer=optimizer,
                criterion=criterion,
                n_epochs=n_epochs,
                device=device,
                model_name=f"{model_class.__name__}_{param_name}_{value}"
            )
            
            # Desactivar la alarma si el entrenamiento termina correctamente
            signal.alarm(0)
            
            # Evaluar modelo
            metrics = evaluate_model(
                model=model,
                test_loader=test_loader,
                criterion=criterion,
                device=device,
                idx2word=text_processor.idx2word
            )
            
            # Guardar resultados
            results[value] = {
                'metrics': metrics,
                'history': history
            }
            
        except TimeoutException:
            print(f"¡Advertencia! El entrenamiento para {param_name}={value} excedió el límite de tiempo de {timeout} segundos.")
            # Continuar con el siguiente valor
            continue
        except Exception as e:
            print(f"Error durante el entrenamiento para {param_name}={value}: {str(e)}")
            # Continuar con el siguiente valor
            continue
    
    # Visualizar resultados si hay suficientes datos
    if len(results) > 1:
        plt.figure(figsize=(15, 10))
        
        # Métricas a visualizar
        metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
        
        for i, metric in enumerate(metrics_to_plot):
            plt.subplot(2, 2, i+1)
            
            values = []
            param_vals = []
            
            for param_value in param_values:
                if param_value in results:
                    values.append(results[param_value]['metrics'][metric])
                    param_vals.append(param_value)
            
            if values:  # Solo graficar si hay valores
                plt.plot(param_vals, values, 'o-', linewidth=2)
                plt.title(f'Impact of {param_name} on {metric.capitalize()}')
                plt.xlabel(param_name)
                plt.ylabel(metric.capitalize())
                plt.grid(True)
                
                # Añadir valores sobre los puntos
                for j, val in enumerate(values):
                    plt.text(param_vals[j], val + 0.01, f'{val:.4f}', ha='center')
        
        plt.tight_layout()
        plt.savefig(f'impact_{param_name}.png')
        plt.close()
    else:
        print(f"No hay suficientes resultados para visualizar el impacto de {param_name}")
    
    return results


In [ ]:
def analyze_examples(model, dataloader, text_processor, device, num_examples=5):
    """
    Analiza ejemplos específicos para entender el comportamiento del modelo
    """
    print(f"\nAnalizando {num_examples} ejemplos específicos para entender el comportamiento del modelo...")
    model.eval()
    examples = []
    
    with torch.no_grad():
        for src, trg in dataloader:
            if len(examples) >= num_examples:
                break
                
            src, trg = src.to(device), trg.to(device)
            output = model(src)
            
            # Obtener predicciones
            predictions = torch.argmax(output, dim=2)
            
            # Analizar cada ejemplo en el batch
            for i in range(src.size(0)):
                if len(examples) >= num_examples:
                    break
                    
                input_text = text_processor.indices_to_text(src[i].cpu().numpy())
                target_text = text_processor.indices_to_text(trg[i].cpu().numpy())
                pred_text = text_processor.indices_to_text(predictions[i].cpu().numpy())
                
                examples.append({
                    'input': input_text,
                    'target': target_text,
                    'prediction': pred_text
                })
    
    # Mostrar ejemplos
    print("\nResultados del análisis de ejemplos específicos:")
    for i, example in enumerate(examples):
        print(f"\nEjemplo {i+1}:")
        print(f"Entrada: {example['input']}")
        print(f"Objetivo: {example['target']}")
        print(f"Predicción: {example['prediction']}")
    
    # Crear visualización interactiva
    fig = go.Figure(data=[
        go.Table(
            header=dict(
                values=['Ejemplo', 'Entrada', 'Objetivo', 'Predicción'],
                fill_color='paleturquoise',
                align='left'
            ),
            cells=dict(
                values=[
                    list(range(1, len(examples) + 1)),
                    [ex['input'] for ex in examples],
                    [ex['target'] for ex in examples],
                    [ex['prediction'] for ex in examples]
                ],
                fill_color='lavender',
                align='left',
                height=30
            )
        )
    ])
    
    fig.update_layout(
        title="Análisis de Ejemplos",
        height=125 * len(examples)
    )
    
    display(fig)
    fig.write_html('example_analysis.html')
    print("Análisis de ejemplos guardado como 'example_analysis.html'")
    
    return examples


In [ ]:
def run_chat_interface(model, text_processor, device, temperature=0.8, beam_size=3):
    """
    Ejecuta una interfaz de chat simple para interactuar con el modelo
    """
    print("\n===== MINI CHAT CON EL MODELO =====")
    print("Escribe un mensaje para conversar con el modelo (o 'salir' para terminar)")
    
    chat_history = []
    
    while True:
        user_input = input("\nTú: ")
        
        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("¡Hasta luego!")
            break
        
        # Generar respuesta
        response = generate_response(
            model=model, 
            text_processor=text_processor, 
            input_text=user_input, 
            device=device,
            temperature=temperature,
            beam_size=beam_size
        )
        
        print(f"Modelo: {response}")
        
        # Guardar historial
        chat_history.append({
            'user': user_input,
            'model': response
        })
    
    # Visualizar historial de chat
    if chat_history:
        print("\nHistorial de chat:")
        for i, exchange in enumerate(chat_history):
            print(f"\nIntercambio {i+1}:")
            print(f"Usuario: {exchange['user']}")
            print(f"Modelo: {exchange['model']}")
        
        # Crear visualización interactiva
        fig = go.Figure(data=[
            go.Table(
                header=dict(
                    values=['Intercambio', 'Usuario', 'Modelo'],
                    fill_color='paleturquoise',
                    align='left'
                ),
                cells=dict(
                    values=[
                        list(range(1, len(chat_history) + 1)),
                        [ex['user'] for ex in chat_history],
                        [ex['model'] for ex in chat_history]
                    ],
                    fill_color='lavender',
                    align='left',
                    height=30
                )
            )
        ])
        
        fig.update_layout(
            title="Historial de Chat",
            height=125 * len(chat_history)
        )
        
        display(fig)
        fig.write_html('chat_history.html')
        print("Historial de chat guardado como 'chat_history.html'")
    
    return chat_history


In [ ]:
class ModelExperiment:
    """
    Clase para gestionar experimentos de modelos de NLP
    """
    def __init__(self, text_processor, train_loader, val_loader, test_loader):
        self.text_processor = text_processor
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = {}
        self.histories = {}
        self.metrics = {}
        self.examples = {}
        
        # Configuración de parámetros
        self.config = {
            'input_dim': text_processor.vocab_size,
            'output_dim': text_processor.vocab_size,
            'emb_dim': 256,
            'hidden_dim': 512,
            'n_layers': 2,
            'n_heads': 8,
            'dropout': 0.3,
            'learning_rate': 0.001,
            'n_epochs': 10
        }
        
        # Criterio de pérdida
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)
        
    def print_header(self, title, width=80):
        """Imprime un encabezado formateado"""
        print("\n" + "=" * width)
        print(f"{title.center(width)}")
        print("=" * width + "\n")
    
    def print_subheader(self, title, width=80):
        """Imprime un subencabezado formateado"""
        print("\n" + "-" * width)
        print(f"{title.center(width)}")
        print("-" * width + "\n")
    
    def print_config(self):
        """Imprime la configuración del experimento"""
        self.print_header("CONFIGURACIÓN DEL EXPERIMENTO")
        
        # Crear una tabla para los parámetros
        from tabulate import tabulate
        
        params = [
            ["Parámetro", "Valor"],
            ["Dimensión de entrada", self.config['input_dim']],
            ["Dimensión de salida", self.config['output_dim']],
            ["Dimensión de embedding", self.config['emb_dim']],
            ["Dimensión oculta", self.config['hidden_dim']],
            ["Número de capas", self.config['n_layers']],
            ["Número de cabezas (Transformer)", self.config['n_heads']],
            ["Dropout", self.config['dropout']],
            ["Tasa de aprendizaje", self.config['learning_rate']],
            ["Número de épocas", self.config['n_epochs']],
            ["Dispositivo", self.device]
        ]
        
        print(tabulate(params, headers="firstrow", tablefmt="fancy_grid"))
        print(f"\nCriterio de pérdida: CrossEntropyLoss (ignorando tokens de padding)")
    
    def initialize_models(self):
        """Inicializa todos los modelos"""
        self.print_header("INICIALIZACIÓN DE MODELOS")
        
        # Modelo RNN simple
        self.print_subheader("Modelo RNN Simple")
        self.models['RNN'] = SimpleRNN(
            input_dim=self.config['input_dim'],
            emb_dim=self.config['emb_dim'],
            hidden_dim=self.config['hidden_dim'],
            output_dim=self.config['output_dim'],
            n_layers=self.config['n_layers'],
            dropout=self.config['dropout']
        ).to(self.device)
        print(f"Modelo RNN inicializado con {sum(p.numel() for p in self.models['RNN'].parameters())} parámetros")
        
        # Modelo LSTM
        self.print_subheader("Modelo LSTM")
        self.models['LSTM'] = LSTM(
            input_dim=self.config['input_dim'],
            emb_dim=self.config['emb_dim'],
            hidden_dim=self.config['hidden_dim'],
            output_dim=self.config['output_dim'],
            n_layers=self.config['n_layers'],
            dropout=self.config['dropout']
        ).to(self.device)
        print(f"Modelo LSTM inicializado con {sum(p.numel() for p in self.models['LSTM'].parameters())} parámetros")
        
        # Modelo GRU
        self.print_subheader("Modelo GRU")
        self.models['GRU'] = GRU(
            input_dim=self.config['input_dim'],
            emb_dim=self.config['emb_dim'],
            hidden_dim=self.config['hidden_dim'],
            output_dim=self.config['output_dim'],
            n_layers=self.config['n_layers'],
            dropout=self.config['dropout']
        ).to(self.device)
        print(f"Modelo GRU inicializado con {sum(p.numel() for p in self.models['GRU'].parameters())} parámetros")
        
        # Modelo Transformer
        self.print_subheader("Modelo Transformer")
        self.models['Transformer'] = TransformerModel(
            input_dim=self.config['input_dim'],
            emb_dim=self.config['emb_dim'],
            hidden_dim=self.config['hidden_dim'],
            output_dim=self.config['output_dim'],
            n_layers=self.config['n_layers'],
            n_heads=self.config['n_heads'],
            dropout=self.config['dropout']
        ).to(self.device)
        print(f"Modelo Transformer inicializado con {sum(p.numel() for p in self.models['Transformer'].parameters())} parámetros")
        
        # Inicializar optimizadores
        self.optimizers = {
            'RNN': optim.Adam(self.models['RNN'].parameters(), lr=self.config['learning_rate']),
            'LSTM': optim.Adam(self.models['LSTM'].parameters(), lr=self.config['learning_rate']),
            'GRU': optim.Adam(self.models['GRU'].parameters(), lr=self.config['learning_rate']),
            'Transformer': optim.Adam(self.models['Transformer'].parameters(), lr=self.config['learning_rate'])
        }
        
        print("\nOptimizadores configurados para todos los modelos (Adam)")
    
    def train_all_models(self):
        """Entrena todos los modelos"""
        self.print_header("ENTRENAMIENTO DE MODELOS")
        
        # Entrenar modelos RNN/LSTM/GRU
        self.print_subheader("Entrenamiento de Modelos RNN/LSTM/GRU")
        
        for model_name in ['RNN', 'LSTM', 'GRU']:
            print(f"\nEntrenando modelo {model_name}...")
            self.models[model_name], self.histories[model_name] = train_model(
                model=self.models[model_name],
                train_loader=self.train_loader,
                val_loader=self.val_loader,
                optimizer=self.optimizers[model_name],
                criterion=self.criterion,
                n_epochs=self.config['n_epochs'],
                device=self.device,
                model_name=model_name
            )
            
            # Visualizar historial de entrenamiento
            plot_training_history(self.histories[model_name], model_name)
        
        # Entrenar modelo Transformer
        self.print_subheader("Entrenamiento del Modelo Transformer")
        
        print("\nEntrenando modelo Transformer...")
        self.models['Transformer'], self.histories['Transformer'] = train_model(
            model=self.models['Transformer'],
            train_loader=self.train_loader,
            val_loader=self.val_loader,
            optimizer=self.optimizers['Transformer'],
            criterion=self.criterion,
            n_epochs=self.config['n_epochs'],
            device=self.device,
            model_name="Transformer"
        )
        
        # Visualizar historial de entrenamiento
        plot_training_history(self.histories['Transformer'], "Transformer")
    
    def evaluate_all_models(self):
        """Evalúa todos los modelos"""
        self.print_header("EVALUACIÓN DE MODELOS")
        
        for model_name in ['RNN', 'LSTM', 'GRU', 'Transformer']:
            self.print_subheader(f"Evaluación del Modelo {model_name}")
            
            self.metrics[model_name] = evaluate_model(
                self.models[model_name], 
                self.test_loader, 
                self.criterion, 
                self.device, 
                self.text_processor.idx2word
            )
            
            # Analizar ejemplos específicos
            print(f"\nAnalizando ejemplos específicos con el modelo {model_name}...")
            self.examples[model_name] = analyze_examples(
                self.models[model_name], 
                self.test_loader, 
                self.text_processor, 
                self.device
            )
    
    def measure_inference_time(self, model, dataloader, device, num_batches=10):
        """
        Mide el tiempo de inferencia promedio por muestra
        """
        model.eval()
        total_time = 0
        total_samples = 0
        
        with torch.no_grad():
            for i, (src, _) in enumerate(dataloader):
                if i >= num_batches:
                    break
                    
                src = src.to(device)
                batch_size = src.size(0)
                
                # Medir tiempo
                start_time = time.time()
                _ = model(src)
                end_time = time.time()
                
                total_time += (end_time - start_time)
                total_samples += batch_size
        
        # Tiempo promedio por muestra
        avg_time = total_time / total_samples
        return avg_time
    
    def compare_models(self):
        """Compara todos los modelos"""
        self.print_header("COMPARACIÓN DE MODELOS")
        
        # Comparar métricas principales
        self.print_subheader("Comparación de Métricas Principales")
        
        compare_models(
            metrics_dict=self.metrics,
            model_names=['RNN', 'LSTM', 'GRU', 'Transformer'],
            metric_names=['accuracy', 'precision', 'recall', 'f1']
        )
        
        # Comparar métricas de NLP
        self.print_subheader("Comparación de Métricas de NLP")
        
        nlp_metrics = ['bleu', 'rouge-1', 'rouge-2', 'rouge-l']
        model_names = ['RNN', 'LSTM', 'GRU', 'Transformer']
        
        for metric in nlp_metrics:
            fig = go.Figure()
            values = [self.metrics[model][metric] for model in model_names]
            
            fig.add_trace(go.Bar(
                x=model_names,
                y=values,
                text=[f'{v:.4f}' for v in values],
                textposition='auto',
                marker_color=['rgba(31, 119, 180, 0.8)', 'rgba(255, 127, 14, 0.8)', 
                             'rgba(44, 160, 44, 0.8)', 'rgba(214, 39, 40, 0.8)']
            ))
            
            fig.update_layout(
                title=f'Comparación de {metric.upper()} entre Modelos',
                xaxis_title='Modelo',
                yaxis_title='Valor',
                template='plotly_white',
                height=500
            )
            
            display(fig)
            fig.write_html(f'comparison_{metric}.html')
        
        # Medir tiempos de inferencia
        self.print_subheader("Comparación de Tiempos de Inferencia")
        
        inference_times = {}
        for model_name in model_names:
            inference_times[model_name] = self.measure_inference_time(
                self.models[model_name], 
                self.test_loader, 
                self.device
            )
        
        # Normalizar tiempos
        min_time = min(inference_times.values())
        relative_times = {model: time/min_time for model, time in inference_times.items()}
        
        # Visualizar tiempos relativos
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=list(relative_times.keys()),
            y=list(relative_times.values()),
            text=[f'{v:.2f}x' for v in relative_times.values()],
            textposition='auto',
            marker_color=['rgba(31, 119, 180, 0.8)', 'rgba(255, 127, 14, 0.8)', 
                         'rgba(44, 160, 44, 0.8)', 'rgba(214, 39, 40, 0.8)']
        ))
        
        fig.update_layout(
            title='Tiempo de Inferencia Relativo (menor es mejor)',
            xaxis_title='Modelo',
            yaxis_title='Tiempo Relativo',
            template='plotly_white',
            height=500
        )
        
        display(fig)
        fig.write_html('inference_times.html')
        
        # Tabla resumen de resultados
        self.print_subheader("Tabla Resumen de Resultados")
        
        from tabulate import tabulate
        
        # Preparar datos para la tabla
        metrics_to_show = ['accuracy', 'f1', 'bleu', 'rouge-l']
        table_data = [["Modelo"] + [m.capitalize() for m in metrics_to_show] + ["Tiempo Rel."]]
        
        for model in model_names:
            row = [model]
            for metric in metrics_to_show:
                row.append(f"{self.metrics[model][metric]:.4f}")
            row.append(f"{relative_times[model]:.2f}x")
            table_data.append(row)
        
        print(tabulate(table_data, headers="firstrow", tablefmt="fancy_grid"))
    
    def analyze_hyperparameters(self):
        """Analiza el impacto de hiperparámetros"""
        self.print_header("ANÁLISIS DE HIPERPARÁMETROS")
        
        # Parámetros fijos
        fixed_params = {
            'emb_dim': self.config['emb_dim'],
            'hidden_dim': self.config['hidden_dim'],
            'output_dim': self.config['output_dim'],
            'n_layers': self.config['n_layers'],
            'dropout': self.config['dropout'],
            'learning_rate': self.config['learning_rate'],
            'n_heads': self.config['n_heads']
        }
        
        # Analizar impacto de la tasa de aprendizaje en LSTM
        self.print_subheader("Impacto de la Tasa de Aprendizaje en LSTM")
        
        lr_values = [0.0001, 0.001, 0.01, 0.1]
        lr_results = analyze_hyperparameters(
            model_class=LSTM,
            train_loader=self.train_loader,
            val_loader=self.val_loader,
            test_loader=self.test_loader,
            text_processor=self.text_processor,
            param_name='learning_rate',
            param_values=lr_values,
            fixed_params=fixed_params,
            n_epochs=5,
            device=self.device,
            timeout=480
        )
        
        # Analizar impacto del número de capas en Transformer
        self.print_subheader("Impacto del Número de Capas en Transformer")
        
        n_layers_values = [1, 2, 3, 4]
        n_layers_transformer_results = analyze_hyperparameters(
            model_class=TransformerModel,
            train_loader=self.train_loader,
            val_loader=self.val_loader,
            test_loader=self.test_loader,
            text_processor=self.text_processor,
            param_name='n_layers',
            param_values=n_layers_values,
            fixed_params=fixed_params,
            n_epochs=5,
            device=self.device,
            timeout=480
        )
        
        # Analizar impacto del número de cabezas de atención en Transformer
        self.print_subheader("Impacto del Número de Cabezas de Atención en Transformer")
        
        n_heads_values = [2, 4, 8, 16]
        n_heads_results = analyze_hyperparameters(
            model_class=TransformerModel,
            train_loader=self.train_loader,
            val_loader=self.val_loader,
            test_loader=self.test_loader,
            text_processor=self.text_processor,
            param_name='n_heads',
            param_values=n_heads_values,
            fixed_params=fixed_params,
            n_epochs=5,
            device=self.device,
            timeout=480
        )
    
    def run_chat_demo(self):
        """Ejecuta una demostración de chat con el mejor modelo"""
        self.print_header("DEMOSTRACIÓN DE CHAT")
        
        # Determinar el mejor modelo basado en F1-score
        best_model_name = max(self.metrics.keys(), key=lambda x: self.metrics[x]['f1'])
        best_model = self.models[best_model_name]
        
        print(f"Usando el modelo {best_model_name} para el chat (mejor F1-score: {self.metrics[best_model_name]['f1']:.4f})")
        
        # Ejecutar interfaz de chat
        try:
            chat_history = run_chat_interface(best_model, self.text_processor, self.device)
            return chat_history
        except NameError:
            print("Función de chat no definida. Implementando una versión simple...")
            
            # Implementación simple de chat
            print("\nEscribe un mensaje para que el modelo responda (o 'salir' para terminar):")
            chat_history = []
            
            while True:
                user_input = input("\nTú: ")
                if user_input.lower() in ['salir', 'exit', 'quit']:
                    break
                
                # Procesar entrada
                input_indices = self.text_processor.text_to_indices(user_input, add_special_tokens=True)
                input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(0).to(self.device)
                
                # Generar respuesta
                with torch.no_grad():
                    output = best_model(input_tensor)
                    predictions = torch.argmax(output, dim=2)
                    response_indices = predictions[0].cpu().numpy()
                    
                # Convertir a texto
                response = self.text_processor.indices_to_text(response_indices)
                
                print(f"Modelo: {response}")
                
                # Guardar en historial
                chat_history.append({"user": user_input, "model": response})
            
            return chat_history
    
    def generate_report(self):
        """Genera un informe final con los resultados"""
        self.print_header("INFORME FINAL DE RESULTADOS")
        
        # Determinar el mejor modelo RNN/LSTM
        rnn_lstm_models = ['RNN', 'LSTM', 'GRU']
        best_rnn_lstm = max(rnn_lstm_models, key=lambda x: self.metrics[x]['f1'])
        
        # Comparar el mejor RNN/LSTM con Transformer
        self.print_subheader("Comparación del Mejor Modelo RNN/LSTM vs Transformer")
        
        # Crear tabla comparativa
        from tabulate import tabulate
        
        metrics_to_compare = ['accuracy', 'f1', 'bleu', 'rouge-l']
        table_data = [["Métrica", best_rnn_lstm, "Transformer", "Diferencia"]]
        
        for metric in metrics_to_compare:
            rnn_value = self.metrics[best_rnn_lstm][metric]
            transformer_value = self.metrics['Transformer'][metric]
            diff = transformer_value - rnn_value
            diff_str = f"{diff:.4f} ({'mejor' if diff > 0 else 'peor'} Transformer)"
            
            table_data.append([
                metric.capitalize(), 
                f"{rnn_value:.4f}", 
                f"{transformer_value:.4f}",
                diff_str
            ])
        
        print(tabulate(table_data, headers="firstrow", tablefmt="fancy_grid"))
        
        # Análisis de componentes clave del Transformer
        self.print_subheader("Análisis de Componentes Clave del Transformer")
        
        components = [
            ["Mecanismo de autoatención", "Permite al modelo atender a diferentes partes de la secuencia de entrada simultáneamente."],
            ["Codificación posicional", "Proporciona información sobre la posición de cada token en la secuencia."],
            ["Arquitectura encoder-decoder", "Permite procesar la entrada y generar la salida de manera eficiente."],
            ["Multi-head attention", "Permite al modelo atender a diferentes representaciones del espacio simultáneamente."]
        ]
        
        print(tabulate(components, headers=["Componente", "Descripción"], tablefmt="fancy_grid"))
        
        # Conclusiones
        self.print_subheader("Conclusiones")
        
        # Comparación de arquitecturas
        print("1. Comparación de arquitecturas:")
        if self.metrics['Transformer']['f1'] > self.metrics[best_rnn_lstm]['f1']:
            print(f"   - El modelo Transformer superó al mejor modelo RNN/LSTM ({best_rnn_lstm}) en términos de F1-score.")
        else:
            print(f"   - El mejor modelo RNN/LSTM ({best_rnn_lstm}) superó al Transformer en términos de F1-score.")

        if self.metrics['Transformer']['bleu'] > self.metrics[best_rnn_lstm]['bleu']:
            print(f"   - El modelo Transformer superó al mejor modelo RNN/LSTM en términos de BLEU score.")
        else:
            print(f"   - El mejor modelo RNN/LSTM superó al Transformer en términos de BLEU score.")
        
        # Impacto de hiperparámetros
        print("\n2. Impacto de hiperparámetros:")
        print("   - Número de capas: Un mayor número de capas puede mejorar el rendimiento hasta cierto punto, pero también aumenta el riesgo de sobreajuste.")
        print("   - Tasa de aprendizaje: Una tasa de aprendizaje adecuada es crucial para la convergencia del modelo.")
        print("   - Número de cabezas de atención (Transformer): Más cabezas permiten capturar diferentes tipos de relaciones en los datos.")
        
        # Ventajas y desventajas
        print("\n3. Ventajas y desventajas:")
        print("   - RNN/LSTM:")
        print("     * Ventajas: Más simples, menos parámetros, eficientes para secuencias cortas.")
        print("     * Desventajas: Dificultad para capturar dependencias a largo plazo, procesamiento secuencial.")
        print("   - Transformer:")
        print("     * Ventajas: Paralelización, mejor captura de dependencias a largo plazo, atención a diferentes partes de la secuencia.")
        print("     * Desventajas: Mayor número de parámetros, requiere más datos para entrenar efectivamente.")
        
        print("\nAnálisis completado. Se han generado gráficos para visualizar los resultados.")

# Función para analizar ejemplos específicos (versión mejorada)
def analyze_examples(model, dataloader, text_processor, device, num_examples=5):
    """
    Analiza ejemplos específicos para entender el comportamiento del modelo
    """
    model.eval()
    examples = []
    
    print(f"Analizando {num_examples} ejemplos específicos para entender el comportamiento del modelo...")
    
    with torch.no_grad():
        for src, trg in dataloader:
            if len(examples) >= num_examples:
                break
                
            src, trg = src.to(device), trg.to(device)
            output = model(src)
            
            # Obtener predicciones
            predictions = torch.argmax(output, dim=2)
            
            # Analizar cada ejemplo en el batch
            for i in range(src.size(0)):
                if len(examples) >= num_examples:
                    break
                    
                # Convertir índices a texto
                input_indices = src[i].cpu().numpy()
                target_indices = trg[i].cpu().numpy()
                pred_indices = predictions[i].cpu().numpy()
                
                input_text = text_processor.indices_to_text(input_indices)
                target_text = text_processor.indices_to_text(target_indices)
                pred_text = text_processor.indices_to_text(pred_indices)
                
                examples.append({
                    'input': input_text,
                    'target': target_text,
                    'prediction': pred_text,
                    'input_raw': input_indices,
                    'target_raw': target_indices,
                    'prediction_raw': pred_indices
                })
    
    # Mostrar ejemplos
    print("\nResultados del análisis de ejemplos específicos:")
    for i, example in enumerate(examples):
        print(f"\nEjemplo {i+1}:")
        print(f"Entrada: {example['input']}")
        print(f"Objetivo: {example['target']}")
        print(f"Predicción: {example['prediction']}")
    
    return examples

# Función principal para ejecutar el experimento completo
def run_nlp_experiment(text_processor, train_loader, val_loader, test_loader):
    """
    Ejecuta el experimento completo de NLP con todos los modelos
    """
    # Crear instancia del experimento
    experiment = ModelExperiment(text_processor, train_loader, val_loader, test_loader)
    
    # Mostrar configuración
    experiment.print_config()
    
    # Inicializar modelos
    experiment.initialize_models()
    
    # Entrenar modelos
    experiment.train_all_models()
    
    # Evaluar modelos
    experiment.evaluate_all_models()
    
    # Comparar modelos
    experiment.compare_models()
    
    # Analizar hiperparámetros
    experiment.analyze_hyperparameters()
    
    # Generar informe final
    experiment.generate_report()
    
    # Preguntar si se desea probar el chat
    response = input("\n¿Deseas probar el modelo en un mini chat? (s/n) ")
    if response.lower() in ['s', 'si', 'sí', 'y', 'yes']:
        experiment.run_chat_demo()
    
    return experiment

# Función para medir tiempos de inferencia
def measure_inference_time(model, dataloader, device, num_batches=10):
    """
    Mide el tiempo de inferencia promedio por muestra
    """
    model.eval()
    total_time = 0
    total_samples = 0
    
    with torch.no_grad():
        for i, (src, _) in enumerate(dataloader):
            if i >= num_batches:
                break
                
            src = src.to(device)
            batch_size = src.size(0)
            
            # Medir tiempo
            start_time = time.time()
            _ = model(src)
            end_time = time.time()
            
            total_time += (end_time - start_time)
            total_samples += batch_size
    
    # Tiempo promedio por muestra
    avg_time = total_time / total_samples
    return avg_time

# Función para analizar hiperparámetros con timeout
def analyze_hyperparameters(model_class, train_loader, val_loader, test_loader, text_processor, 
                           param_name, param_values, fixed_params, n_epochs, device, timeout=None):
    """
    Analiza el impacto de un hiperparámetro específico con opción de timeout
    """
    results = {}
    
    for value in param_values:
        print(f"\nEntrenando modelo con {param_name}={value}")
        
        # Crear modelo con el valor actual del hiperparámetro
        params = fixed_params.copy()
        params[param_name] = value
        
        if model_class.__name__ == 'TransformerModel':
            model = model_class(
                input_dim=text_processor.vocab_size,
                emb_dim=params['emb_dim'],
                hidden_dim=params['hidden_dim'],
                output_dim=params['output_dim'],
                n_layers=params['n_layers'],
                n_heads=params['n_heads'],
                dropout=params['dropout']
            ).to(device)
        else:
            model = model_class(
                input_dim=text_processor.vocab_size,
                emb_dim=params['emb_dim'],
                hidden_dim=params['hidden_dim'],
                output_dim=params['output_dim'],
                n_layers=params['n_layers'],
                dropout=params['dropout']
            ).to(device)
        
        # Crear optimizador
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
        
        # Criterio de pérdida
        criterion = nn.CrossEntropyLoss(ignore_index=0)
        
        # Entrenar modelo con timeout
        start_time = time.time()
        try:
            if timeout:
                # Configurar un manejador de señal para timeout
                import signal
                
                class TimeoutException(Exception):
                    pass
                
                def timeout_handler(signum, frame):
                    raise TimeoutException("Entrenamiento interrumpido por timeout")
                
                # Configurar el manejador de señal
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(timeout)  # Timeout en segundos
            
            # Entrenar modelo
            model, history = train_model(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                optimizer=optimizer,
                criterion=criterion,
                n_epochs=n_epochs,
                device=device,
                model_name=f"{model_class.__name__}_{param_name}_{value}"
            )
            
            if timeout:
                # Desactivar la alarma
                signal.alarm(0)
            
        except TimeoutException:
            print(f"Entrenamiento interrumpido por timeout ({timeout} segundos)")
            # Usar el modelo en su estado actual
            history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
        except Exception as e:
            print(f"Error durante el entrenamiento: {e}")
            continue
        
        # Evaluar modelo
        try:
            metrics = evaluate_model(
                model=model,
                test_loader=test_loader,
                criterion=criterion,
                device=device,
                idx2word=text_processor.idx2word
            )
        except Exception as e:
            print(f"Error durante la evaluación: {e}")
            metrics = {
                'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0,
                'bleu': 0, 'rouge-1': 0, 'rouge-2': 0, 'rouge-l': 0
            }
        
        # Guardar resultados
        results[value] = {
            'metrics': metrics,
            'history': history,
            'training_time': time.time() - start_time
        }
    
    # Visualizar resultados
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        
        # Métricas a visualizar
        metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
        
        # Crear subplots
        fig = make_subplots(rows=2, cols=2, subplot_titles=[m.capitalize() for m in metrics_to_plot])
        
        for i, metric in enumerate(metrics_to_plot):
            row, col = i // 2 + 1, i % 2 + 1
            
            values = [results[param_value]['metrics'][metric] for param_value in param_values]
            
            fig.add_trace(
                go.Scatter(
                    x=param_values, 
                    y=values, 
                    mode='lines+markers',
                    name=metric.capitalize(),
                    text=[f'{v:.4f}' for v in values],
                    hoverinfo='text+x'
                ),
                row=row, col=col
            )
            
            fig.update_xaxes(title_text=param_name, row=row, col=col)
            fig.update_yaxes(title_text=metric.capitalize(), row=row, col=col)
        
        fig.update_layout(
            title=f'Impacto de {param_name} en el Rendimiento del Modelo',
            height=600,
            width=900,
            showlegend=False,
            template='plotly_white'
        )
        
        # Mostrar figura
        fig.show()
        
        # Guardar figura
        fig.write_html(f'impact_{param_name}.html')
        
    except Exception as e:
        print(f"Error al visualizar resultados: {e}")
        
        # Alternativa con matplotlib
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(15, 10))
        
        # Métricas a visualizar
        metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
        
        for i, metric in enumerate(metrics_to_plot):
            plt.subplot(2, 2, i+1)
            
            values = [results[param_value]['metrics'][metric] for param_value in param_values]
            
            plt.plot(param_values, values, 'o-', linewidth=2)
            plt.title(f'Impact of {param_name} on {metric.capitalize()}')
            plt.xlabel(param_name)
            plt.ylabel(metric.capitalize())
            plt.grid(True)
            
            # Añadir valores sobre los puntos
            for j, val in enumerate(values):
                plt.text(param_values[j], val + 0.01, f'{val:.4f}', ha='center')
        
        plt.tight_layout()
        plt.savefig(f'impact_{param_name}.png')
        plt.close()
    
    return results

# Función para entrenar un modelo
def train_model(model, train_loader, val_loader, optimizer, criterion, n_epochs, device, model_name):
    """
    Entrena un modelo y guarda el mejor modelo basado en la pérdida de validación
    """
    best_valid_loss = float('inf')
    train_losses = []
    train_accs = []
    valid_losses = []
    valid_accs = []
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        # Entrenar una época
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
        
        # Evaluar en conjunto de validación
        valid_loss, valid_acc, _, _ = evaluate(model, val_loader, criterion, device)
        
        # Guardar métricas
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)
        
        # Guardar el mejor modelo
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f'{model_name}_best.pt')
        
        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc*100:.2f}%')
    
    # Cargar el mejor modelo
    try:
        model.load_state_dict(torch.load(f'{model_name}_best.pt'))
    except:
        print(f"No se pudo cargar el mejor modelo para {model_name}, usando el modelo actual")
    
    # Devolver historiales para visualización
    history = {
        'train_loss': train_losses,
        'train_acc': train_accs,
        'val_loss': valid_losses,
        'val_acc': valid_accs
    }
    
    return model, history

# Función para evaluar un modelo
def evaluate_model(model, test_loader, criterion, device, idx2word):
    """
    Evalúa un modelo en el conjunto de prueba y calcula métricas adicionales
    """
    print("Evaluando modelo en el conjunto de prueba...")
    test_loss, test_acc, all_preds, all_trgs = evaluate(model, test_loader, criterion, device)
    
    print(f'Pérdida de prueba: {test_loss:.4f} | Precisión de prueba: {test_acc*100:.2f}%')
    
    # Calcular métricas adicionales
    print("Calculando métricas adicionales...")
    metrics = calculate_metrics(all_preds, all_trgs, idx2word)
    
    print(f"Resumen de métricas:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")
    
    return metrics

# Función para visualizar el historial de entrenamiento
def plot_training_history(history, model_name):
    """
    Visualiza el historial de entrenamiento
    """
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        
        # Crear subplots
        fig = make_subplots(rows=1, cols=2, subplot_titles=["Pérdida", "Precisión"])
        
        # Añadir trazas para pérdida
        fig.add_trace(
            go.Scatter(
                x=list(range(1, len(history['train_loss'])+1)),
                y=history['train_loss'],
                mode='lines+markers',
                name='Entrenamiento',
                line=dict(color='blue')
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=list(range(1, len(history['val_loss'])+1)),
                y=history['val_loss'],
                mode='lines+markers',
                name='Validación',
                line=dict(color='red')
            ),
            row=1, col=1
        )
        
        # Añadir trazas para precisión
        fig.add_trace(
            go.Scatter(
                x=list(range(1, len(history['train_acc'])+1)),
                y=history['train_acc'],
                mode='lines+markers',
                name='Entrenamiento',
                line=dict(color='blue'),
                showlegend=False
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=list(range(1, len(history['val_acc'])+1)),
                y=history['val_acc'],
                mode='lines+markers',
                name='Validación',
                line=dict(color='red'),
                showlegend=False
            ),
            row=1, col=2
        )
        
        # Actualizar diseño
        fig.update_layout(
            title=f'Historial de Entrenamiento del Modelo {model_name}',
            height=400,
            width=900,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            template='plotly_white'
        )
        
        # Actualizar ejes
        fig.update_xaxes(title_text="Época", row=1, col=1)
        fig.update_xaxes(title_text="Época", row=1, col=2)
        fig.update_yaxes(title_text="Pérdida", row=1, col=1)
        fig.update_yaxes(title_text="Precisión", row=1, col=2)
        
        # Mostrar figura
        fig.show()
        
        # Guardar figura
        fig.write_html(f'{model_name}_history.html')
        
    except Exception as e:
        print(f"Error al visualizar con Plotly: {e}")
        
        # Alternativa con matplotlib
        plt.figure(figsize=(12, 5))
        
        # Gráfico de pérdida
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train')
        plt.plot(history['val_loss'], label='Validation')
        plt.title('Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        # Gráfico de precisión
        plt.subplot(1, 2, 2)
        plt.plot(history['train_acc'], label='Train')
        plt.plot(history['val_acc'], label='Validation')
        plt.title('Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(f'{model_name}_history.png')
        plt.close()

# Función para comparar modelos
def compare_models(metrics_dict, model_names, metric_names):
    """
    Compara diferentes modelos según varias métricas
    """
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        
        # Crear subplots
        fig = make_subplots(rows=2, cols=2, subplot_titles=[m.capitalize() for m in metric_names])
        
        # Colores para cada modelo
        colors = ['rgba(31, 119, 180, 0.8)', 'rgba(255, 127, 14, 0.8)', 
                 'rgba(44, 160, 44, 0.8)', 'rgba(214, 39, 40, 0.8)']
        
        for i, metric in enumerate(metric_names):
            row, col = i // 2 + 1, i % 2 + 1
            
            values = [metrics_dict[model][metric] for model in model_names]
            
            fig.add_trace(
                go.Bar(
                    x=model_names,
                    y=values,
                    text=[f'{v:.4f}' for v in values],
                    textposition='auto',
                    marker_color=colors[:len(model_names)]
                ),
                row=row, col=col
            )
            
            fig.update_yaxes(title_text=metric.capitalize(), row=row, col=col)
        
        fig.update_layout(
            title='Comparación de Modelos por Métricas',
            height=600,
            width=900,
            showlegend=False,
            template='plotly_white'
        )
        
        # Mostrar figura
        fig.show()
        
        # Guardar figura
        fig.write_html('model_comparison.html')
        
    except Exception as e:
        print(f"Error al visualizar con Plotly: {e}")
        
        # Alternativa con matplotlib
        plt.figure(figsize=(15, 10))
        
        for i, metric in enumerate(metric_names):
            plt.subplot(2, 2, i+1)
            values = [metrics_dict[model][metric] for model in model_names]
            
            # Crear gráfico de barras
            bars = plt.bar(model_names, values)
            
            # Añadir valores sobre las barras
            for bar in bars:
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                        f'{height:.4f}', ha='center', va='bottom')
            
            plt.title(metric.capitalize())
            plt.ylabel('Value')
            plt.ylim(0, max(values) * 1.2)  # Ajustar límite vertical
        
        plt.tight_layout()
        plt.savefig('model_comparison.png')
        plt.close()

# Función para calcular métricas adicionales
def calculate_metrics(predictions, targets, idx2word):
    """
    Calcula métricas adicionales como F1, precisión, recall y BLEU/ROUGE
    """
    print("Calculando métricas de evaluación...")
    
    # Convertir índices a palabras
    pred_texts = []
    target_texts = []
    
    for pred, target in zip(predictions, targets):
        # Filtrar tokens especiales (0=PAD, 1=UNK, 2=SOS, 3=EOS)
        pred_text = [idx2word.get(idx, '<UNK>') for idx in pred if idx > 3]
        target_text = [idx2word.get(idx, '<UNK>') for idx in target if idx > 3]
        
        pred_texts.append(pred_text)
        target_texts.append([target_text])  # BLEU espera una lista de referencias
    
    # Calcular BLEU
    print("Calculando BLEU score...")
    try:
        smoothie = SmoothingFunction().method1
        bleu_score = corpus_bleu(target_texts, pred_texts, smoothing_function=smoothie)
    except Exception as e:
        print(f"Error al calcular BLEU: {e}")
        bleu_score = 0
    
    # Calcular ROUGE
    print("Calculando métricas ROUGE...")
    try:
        rouge = Rouge()
        
        # Convertir listas de tokens a strings
        pred_strings = [' '.join(pred) for pred in pred_texts]
        target_strings = [' '.join(target[0]) for target in target_texts]
        
        # Asegurarse de que no hay strings vacíos
        valid_pairs = [(p, t) for p, t in zip(pred_strings, target_strings) if p and t]
        
        if valid_pairs:
            pred_valid, target_valid = zip(*valid_pairs)
            rouge_scores = rouge.get_scores(pred_valid, target_valid, avg=True)
            rouge_1 = rouge_scores['rouge-1']['f']
            rouge_2 = rouge_scores['rouge-2']['f']
            rouge_l = rouge_scores['rouge-l']['f']
        else:
            rouge_1 = rouge_2 = rouge_l = 0
            
        print(f"ROUGE-1: {rouge_1:.4f}, ROUGE-2: {rouge_2:.4f}, ROUGE-L: {rouge_l:.4f}")
    except Exception as e:
        print(f"Error al calcular ROUGE: {e}")
        rouge_1 = rouge_2 = rouge_l = 0
    
    # Calcular precisión, recall y F1 (para tareas de clasificación)
    print("Calculando métricas de clasificación...")
    try:
        # Aplanar todas las predicciones y targets
        all_preds = []
        all_targets = []
        
        for pred, target in zip(predictions, targets):
            # Filtrar tokens de padding
            mask = target != 0
            all_preds.extend(pred[mask])
            all_targets.extend(target[mask])
        
        precision = precision_score(all_targets, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_targets, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
        accuracy = accuracy_score(all_targets, all_preds)
        
        print(f"Precisión: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}")
    except Exception as e:
        print(f"Error al calcular métricas de clasificación: {e}")
        precision = recall = f1 = accuracy = 0
    
    return {
        'bleu': bleu_score,
        'rouge-1': rouge_1,
        'rouge-2': rouge_2,
        'rouge-l': rouge_l,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# Función para entrenar una época
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    total_samples = 0
    
    for batch_idx, (src, trg) in enumerate(tqdm(dataloader, desc="Training")):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        output = model(src)
        
        # Reshape para calcular pérdida
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        trg = trg.view(-1)
        
        # Calcular pérdida
        loss = criterion(output, trg)
        
        # Backward pass
        loss.backward()
        
        # Actualizar pesos
        optimizer.step()
        
        # Calcular precisión
        _, predicted = torch.max(output, 1)
        correct = (predicted == trg).float()
        mask = (trg != 0).float()  # Ignorar padding
        correct = (correct * mask).sum().item()
        total = mask.sum().item()
        
        # Actualizar métricas
        epoch_loss += loss.item() * src.size(0)
        epoch_acc += correct
        total_samples += total
    
    return epoch_loss / len(dataloader.dataset), epoch_acc / total_samples

# Función para evaluar
def evaluate(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    total_samples = 0
    
    all_preds = []
    all_trgs = []
    
    with torch.no_grad():
        for batch_idx, (src, trg) in enumerate(tqdm(dataloader, desc="Evaluating")):
            src, trg = src.to(device), trg.to(device)
            
            # Forward pass
            output = model(src)
            
            # Reshape para calcular pérdida
            output_dim = output.shape[-1]
            output_flat = output.view(-1, output_dim)
            trg_flat = trg.view(-1)
            
            # Calcular pérdida
            loss = criterion(output_flat, trg_flat)
            
            # Calcular precisión
            _, predicted = torch.max(output_flat, 1)
            correct = (predicted == trg_flat).float()
            mask = (trg_flat != 0).float()  # Ignorar padding
            correct = (correct * mask).sum().item()
            total = mask.sum().item()
            
            # Actualizar métricas
            epoch_loss += loss.item() * src.size(0)
            epoch_acc += correct
            total_samples += total
            
            # Guardar predicciones y targets para calcular métricas adicionales
            for i in range(src.size(0)):
                pred_seq = torch.argmax(output[i], dim=1).cpu().numpy()
                trg_seq = trg[i].cpu().numpy()
                
                # Filtrar padding
                mask = trg_seq != 0
                pred_seq_filtered = pred_seq[mask]
                trg_seq_filtered = trg_seq[mask]
                
                all_preds.append(pred_seq_filtered)
                all_trgs.append(trg_seq_filtered)
    
    return epoch_loss / len(dataloader.dataset), epoch_acc / total_samples, all_preds, all_trgs

# Función para ejecutar el experimento completo
def run_nlp_experiment(text_processor, train_loader, val_loader, test_loader):
    """
    Ejecuta el experimento completo de NLP con todos los modelos
    """
    # Crear instancia del experimento
    experiment = ModelExperiment(text_processor, train_loader, val_loader, test_loader)
    
    # Mostrar configuración
    experiment.print_config()
    
    # Inicializar modelos
    experiment.initialize_models()
    
    # Entrenar modelos
    experiment.train_all_models()
    
    # Evaluar modelos
    experiment.evaluate_all_models()
    
    # Comparar modelos
    experiment.compare_models()
    
    # Analizar hiperparámetros
    experiment.analyze_hyperparameters()
    
    # Generar informe final
    experiment.generate_report()
    
    # Preguntar si se desea probar el chat
    response = input("\n¿Deseas probar el modelo en un mini chat? (s/n) ")
    if response.lower() in ['s', 'si', 'sí', 'y', 'yes']:
        experiment.run_chat_demo()
    
    return experiment


In [ ]:
# ======= ANÁLISIS COMPARATIVO FINAL =======
print("\n===== ANÁLISIS COMPARATIVO FINAL =====")

# Comparar tiempos de inferencia
def measure_inference_time(model, dataloader, device, num_batches=10):
    """
    Mide el tiempo promedio de inferencia por muestra
    """
    print(f"Midiendo tiempo de inferencia para {model.__class__.__name__}...")
    model.eval()
    total_time = 0
    total_samples = 0
    
    with torch.no_grad():
        for i, (src, _) in enumerate(dataloader):
            if i >= num_batches:
                break
                
            src = src.to(device)
            batch_size = src.size(0)
            
            # Medir tiempo
            start_time = time.time()
            _ = model(src)
            end_time = time.time()
            
            total_time += (end_time - start_time)
            total_samples += batch_size
    
    # Tiempo promedio por muestra
    avg_time = total_time / total_samples
    print(f"Tiempo promedio de inferencia por muestra: {avg_time*1000:.2f} ms")
    return avg_time



===== ANÁLISIS COMPARATIVO FINAL =====


In [ ]:
experiment = run_nlp_experiment(text_processor, train_loader, val_loader, test_loader)


                         CONFIGURACIÓN DEL EXPERIMENTO                          

╒═════════════════════════════════╤═════════╕
│ Parámetro                       │ Valor   │
╞═════════════════════════════════╪═════════╡
│ Dimensión de entrada            │ 10000   │
├─────────────────────────────────┼─────────┤
│ Dimensión de salida             │ 10000   │
├─────────────────────────────────┼─────────┤
│ Dimensión de embedding          │ 256     │
├─────────────────────────────────┼─────────┤
│ Dimensión oculta                │ 512     │
├─────────────────────────────────┼─────────┤
│ Número de capas                 │ 2       │
├─────────────────────────────────┼─────────┤
│ Número de cabezas (Transformer) │ 8       │
├─────────────────────────────────┼─────────┤
│ Dropout                         │ 0.3     │
├─────────────────────────────────┼─────────┤
│ Tasa de aprendizaje             │ 0.001   │
├─────────────────────────────────┼─────────┤
│ Número de épocas                │ 10     

NameError: name 'PositionalEncoding' is not defined

In [ ]:
print("\nMidiendo tiempos de inferencia para todos los modelos...")
rnn_time = measure_inference_time(rnn_model, test_loader, device)
lstm_time = measure_inference_time(lstm_model, test_loader, device)
gru_time = measure_inference_time(gru_model, test_loader, device)
transformer_time = measure_inference_time(transformer_model, test_loader, device)

# Normalizar tiempos (relativo al más rápido)
min_time = min(rnn_time, lstm_time, gru_time, transformer_time)
relative_times = {
    'RNN': rnn_time / min_time,
    'LSTM': lstm_time / min_time,
    'GRU': gru_time / min_time,
    'Transformer': transformer_time / min_time
}

print(f"\nTiempos de inferencia relativos (menor es mejor):")
for model_name, rel_time in relative_times.items():
    print(f"{model_name}: {rel_time:.2f}x")

# Visualizar tiempos de inferencia
print("\nVisualizando tiempos de inferencia relativos...")
fig = go.Figure()
fig.add_trace(go.Bar(
    x=list(relative_times.keys()),
    y=list(relative_times.values()),
    text=[f'{v:.2f}x' for v in relative_times.values()],
    textposition='auto'
))

fig.update_layout(
    title='Tiempo de inferencia relativo (menor es mejor)',
    xaxis_title='Modelo',
    yaxis_title='Tiempo relativo',
    template='plotly_white'
)

display(fig)
fig.write_html('inference_times.html')

# Resumen final de resultados
print("\n===== RESUMEN FINAL DE RESULTADOS =====")
print("\nMétricas de evaluación por modelo:")

# Crear tabla interactiva con todas las métricas
metrics_table = go.Figure(data=[go.Table(
    header=dict(
        values=['Métrica'] + model_names,
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=[
            ['Accuracy', 'Precision', 'Recall', 'F1', 'BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'Tiempo relativo'],
            [f"{all_metrics['RNN']['accuracy']:.4f}", f"{all_metrics['RNN']['precision']:.4f}", 
             f"{all_metrics['RNN']['recall']:.4f}", f"{all_metrics['RNN']['f1']:.4f}", 
             f"{all_metrics['RNN']['bleu']:.4f}", f"{all_metrics['RNN']['rouge-1']:.4f}", 
             f"{all_metrics['RNN']['rouge-2']:.4f}", f"{all_metrics['RNN']['rouge-l']:.4f}", 
             f"{relative_times['RNN']:.2f}x"],
            [f"{all_metrics['LSTM']['accuracy']:.4f}", f"{all_metrics['LSTM']['precision']:.4f}", 
             f"{all_metrics['LSTM']['recall']:.4f}", f"{all_metrics['LSTM']['f1']:.4f}", 
             f"{all_metrics['LSTM']['bleu']:.4f}", f"{all_metrics['LSTM']['rouge-1']:.4f}", 
             f"{all_metrics['LSTM']['rouge-2']:.4f}", f"{all_metrics['LSTM']['rouge-l']:.4f}", 
             f"{relative_times['LSTM']:.2f}x"],
            [f"{all_metrics['GRU']['accuracy']:.4f}", f"{all_metrics['GRU']['precision']:.4f}", 
             f"{all_metrics['GRU']['recall']:.4f}", f"{all_metrics['GRU']['f1']:.4f}", 
             f"{all_metrics['GRU']['bleu']:.4f}", f"{all_metrics['GRU']['rouge-1']:.4f}", 
             f"{all_metrics['GRU']['rouge-2']:.4f}", f"{all_metrics['GRU']['rouge-l']:.4f}", 
             f"{relative_times['GRU']:.2f}x"],
            [f"{all_metrics['Transformer']['accuracy']:.4f}", f"{all_metrics['Transformer']['precision']:.4f}", 
             f"{all_metrics['Transformer']['recall']:.4f}", f"{all_metrics['Transformer']['f1']:.4f}", 
             f"{all_metrics['Transformer']['bleu']:.4f}", f"{all_metrics['Transformer']['rouge-1']:.4f}", 
             f"{all_metrics['Transformer']['rouge-2']:.4f}", f"{all_metrics['Transformer']['rouge-l']:.4f}", 
             f"{relative_times['Transformer']:.2f}x"]
        ],
        fill_color='lavender',
        align='left'
    )
)])

metrics_table.update_layout(
    title="Resumen de Métricas por Modelo",
    height=400
)

display(metrics_table)
metrics_table.write_html('metrics_summary.html')

# Seleccionar el mejor modelo RNN/LSTM basado en F1-score
best_rnn_lstm_model = max(['RNN', 'LSTM', 'GRU'], key=lambda x: all_metrics[x]['f1'])
print(f"\nMejor modelo RNN/LSTM: {best_rnn_lstm_model} (F1: {all_metrics[best_rnn_lstm_model]['f1']:.4f})")

# Comparar el mejor modelo RNN/LSTM con Transformer
print("\nComparación del mejor modelo RNN/LSTM vs Transformer:")
print(f"F1-score - {best_rnn_lstm_model}: {all_metrics[best_rnn_lstm_model]['f1']:.4f}, Transformer: {all_metrics['Transformer']['f1']:.4f}")
print(f"BLEU - {best_rnn_lstm_model}: {all_metrics[best_rnn_lstm_model]['bleu']:.4f}, Transformer: {all_metrics['Transformer']['bleu']:.4f}")
print(f"ROUGE-L - {best_rnn_lstm_model}: {all_metrics[best_rnn_lstm_model]['rouge-l']:.4f}, Transformer: {all_metrics['Transformer']['rouge-l']:.4f}")
print(f"Tiempo relativo - {best_rnn_lstm_model}: {relative_times[best_rnn_lstm_model]:.2f}x, Transformer: {relative_times['Transformer']:.2f}x")

# Visualizar comparación final entre el mejor RNN/LSTM y Transformer
print("\nVisualizando comparación final entre el mejor modelo RNN/LSTM y Transformer...")
final_metrics = ['accuracy', 'f1', 'bleu', 'rouge-l']
final_models = [best_rnn_lstm_model, 'Transformer']

for metric in final_metrics:
    fig = go.Figure()
    values = [all_metrics[model][metric] for model in final_models]
    
    fig.add_trace(go.Bar(
        x=final_models,
        y=values,
        text=[f'{v:.4f}' for v in values],
        textposition='auto'
    ))
    
    fig.update_layout(
        title=f'Comparación de {metric.capitalize()} - {best_rnn_lstm_model} vs Transformer',
        xaxis_title='Modelo',
        yaxis_title=metric.capitalize(),
        template='plotly_white'
    )
    
    display(fig)
    fig.write_html(f'final_comparison_{metric}.html')

In [ ]:
# Análisis de componentes clave del Transformer
print("\nAnálisis de componentes clave del Transformer:")
print("1. Mecanismo de autoatención: Permite al modelo atender a diferentes partes de la secuencia de entrada simultáneamente.")
print("2. Codificación posicional: Proporciona información sobre la posición de cada token en la secuencia.")
print("3. Arquitectura encoder-decoder: Permite procesar la entrada y generar la salida de manera eficiente.")
print("4. Multi-head attention: Permite al modelo atender a diferentes representaciones del espacio simultáneamente.")

In [ ]:
# Conclusiones
print("\n===== CONCLUSIONES =====")
print("1. Comparación de arquitecturas:")
if all_metrics['Transformer']['f1'] > all_metrics[best_rnn_lstm_model]['f1']:
    print(f"   - El modelo Transformer superó al mejor modelo RNN/LSTM ({best_rnn_lstm_model}) en términos de F1-score.")
else:
    print(f"   - El mejor modelo RNN/LSTM ({best_rnn_lstm_model}) superó al Transformer en términos de F1-score.")

if all_metrics['Transformer']['bleu'] > all_metrics[best_rnn_lstm_model]['bleu']:
    print(f"   - El modelo Transformer superó al mejor modelo RNN/LSTM en términos de BLEU score.")
else:
    print(f"   - El mejor modelo RNN/LSTM superó al Transformer en términos de BLEU score.")

if relative_times['Transformer'] < relative_times[best_rnn_lstm_model]:
    print(f"   - El modelo Transformer fue más rápido en inferencia que el mejor modelo RNN/LSTM.")
else:
    print(f"   - El mejor modelo RNN/LSTM fue más rápido en inferencia que el Transformer.")

print("\n2. Impacto de hiperparámetros:")
print("   - Número de capas: Un mayor número de capas puede mejorar el rendimiento hasta cierto punto, pero también aumenta el riesgo de sobreajuste.")
print("   - Tasa de aprendizaje: Una tasa de aprendizaje adecuada es crucial para la convergencia del modelo.")
print("   - Número de cabezas de atención (Transformer): Más cabezas permiten capturar diferentes tipos de relaciones en los datos.")

print("\n3. Ventajas y desventajas:")
print("   - RNN/LSTM:")
print("     * Ventajas: Más simples, menos parámetros, eficientes para secuencias cortas.")
print("     * Desventajas: Dificultad para capturar dependencias a largo plazo, procesamiento secuencial.")
print("   - Transformer:")
print("     * Ventajas: Paralelización, mejor captura de dependencias a largo plazo, atención a diferentes partes de la secuencia.")
print("     * Desventajas: Mayor número de parámetros, requiere más datos para entrenar efectivamente.")

print("\nAnálisis completado. Se han generado gráficos interactivos para visualizar los resultados.")

In [ ]:
# Mini chat para probar el modelo
print("\n¿Deseas probar el modelo en un mini chat? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    # Usar el mejor modelo según las métricas
    best_model_name = max(['RNN', 'LSTM', 'GRU', 'Transformer'], 
                          key=lambda x: all_metrics[x]['f1'])
    
    print(f"Usando el modelo {best_model_name} para el chat...")
    
    if best_model_name == 'RNN':
        chat_model = rnn_model
    elif best_model_name == 'LSTM':
        chat_model = lstm_model
    elif best_model_name == 'GRU':
        chat_model = gru_model
    else:
        chat_model = transformer_model
    
    # Ejecutar interfaz de chat
    chat_history = run_chat_interface(chat_model, text_processor, device)

In [ ]:
# Generar informe final en HTML
def generate_final_report(all_metrics, relative_times, model_names, best_rnn_lstm_model):
    """
    Genera un informe final en HTML con todos los resultados
    """
    print("\nGenerando informe final en HTML...")
    
    html_content = f"""
    <!DOCTYPE html>
    <html lang="es">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Informe Final - Comparación de Modelos NLP</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
            }}
            h1, h2, h3 {{
                color: #2c3e50;
            }}
            h1 {{
                text-align: center;
                border-bottom: 2px solid #3498db;
                padding-bottom: 10px;
            }}
            h2 {{
                border-bottom: 1px solid #bdc3c7;
                padding-bottom: 5px;
                margin-top: 30px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 20px 0;
            }}
            th, td {{
                padding: 12px 15px;
                text-align: left;
                border-bottom: 1px solid #ddd;
            }}
            th {{
                background-color: #f2f2f2;
                font-weight: bold;
            }}
            tr:hover {{
                background-color: #f5f5f5;
            }}
            .highlight {{
                background-color: #e8f4f8;
                font-weight: bold;
            }}
            .container {{
                display: flex;
                flex-wrap: wrap;
                justify-content: space-between;
            }}
            .chart-container {{
                width: 48%;
                margin-bottom: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                padding: 15px;
                border-radius: 5px;
            }}
            .full-width {{
                width: 100%;
            }}
            .conclusion {{
                background-color: #f9f9f9;
                padding: 15px;
                border-left: 4px solid #3498db;
                margin: 20px 0;
            }}
            .footer {{
                text-align: center;
                margin-top: 50px;
                padding-top: 20px;
                border-top: 1px solid #ddd;
                color: #7f8c8d;
            }}
            .advantage {{
                color: #27ae60;
            }}
            .disadvantage {{
                color: #e74c3c;
            }}
        </style>
    </head>
    <body>
        <h1>Informe Final: Comparación de Modelos RNN/LSTM y Transformer para NLP</h1>
        
        <h2>1. Resumen Ejecutivo</h2>
        <p>
            Este informe presenta un análisis comparativo entre diferentes arquitecturas de redes neuronales
            para el procesamiento de lenguaje natural (NLP): RNN simple, LSTM, GRU y Transformer.
            Se evaluaron estos modelos en términos de precisión, métricas específicas de NLP y eficiencia computacional.
        </p>
        
        <h2>2. Configuración Experimental</h2>
        <p>
            <strong>Parámetros de los modelos:</strong>
            <ul>
                <li>Dimensión de entrada/salida: {INPUT_DIM}</li>
                <li>Dimensión de embedding: {EMB_DIM}</li>
                <li>Dimensión oculta: {HIDDEN_DIM}</li>
                <li>Número de capas: {N_LAYERS}</li>
                <li>Número de cabezas (Transformer): {N_HEADS}</li>
                <li>Dropout: {DROPOUT}</li>
                <li>Tasa de aprendizaje: {LEARNING_RATE}</li>
                <li>Épocas de entrenamiento: {N_EPOCHS}</li>
            </ul>
        </p>
        
        <h2>3. Resultados Comparativos</h2>
        
        <h3>3.1. Tabla de Métricas</h3>
        <table>
            <tr>
                <th>Métrica</th>
    """
    
    # Añadir encabezados de columnas para cada modelo
    for model in model_names:
        html_content += f"<th>{model}</th>"
    
    html_content += """
            </tr>
    """
    
    # Añadir filas para cada métrica
    metrics_list = ['accuracy', 'precision', 'recall', 'f1', 'bleu', 'rouge-1', 'rouge-2', 'rouge-l']
    metrics_display = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L']
    
    for i, metric in enumerate(metrics_list):
        html_content += f"""
            <tr>
                <td>{metrics_display[i]}</td>
        """
        
        # Encontrar el mejor valor para esta métrica
        best_value = max([all_metrics[model][metric] for model in model_names])
        
        for model in model_names:
            value = all_metrics[model][metric]
            if value == best_value:
                html_content += f"<td class='highlight'>{value:.4f}</td>"
            else:
                html_content += f"<td>{value:.4f}</td>"
        
        html_content += """
            </tr>
        """
    
    # Añadir fila para tiempo de inferencia
    html_content += """
            <tr>
                <td>Tiempo de inferencia relativo</td>
    """
    
    # Encontrar el mejor tiempo (menor valor)
    best_time = min([relative_times[model] for model in model_names])
    
    for model in model_names:
        time_value = relative_times[model]
        if time_value == best_time:
            html_content += f"<td class='highlight'>{time_value:.2f}x</td>"
        else:
            html_content += f"<td>{time_value:.2f}x</td>"
    
    html_content += """
            </tr>
        </table>
        
        <h3>3.2. Visualizaciones</h3>
        
        <div class="container">
            <div class="chart-container">
                <h4>Comparación de Accuracy</h4>
                <img src="comparison_accuracy.png" alt="Comparación de Accuracy" width="100%">
            </div>
            
            <div class="chart-container">
                <h4>Comparación de F1-Score</h4>
                <img src="comparison_f1.png" alt="Comparación de F1-Score" width="100%">
            </div>
            
            <div class="chart-container">
                <h4>Comparación de BLEU</h4>
                <img src="comparison_bleu.png" alt="Comparación de BLEU" width="100%">
            </div>
            
            <div class="chart-container">
                <h4>Comparación de ROUGE-L</h4>
                <img src="comparison_rouge-l.png" alt="Comparación de ROUGE-L" width="100%">
            </div>
            
            <div class="chart-container full-width">
                <h4>Tiempos de Inferencia Relativos</h4>
                <img src="inference_times.png" alt="Tiempos de Inferencia" width="100%">
            </div>
        </div>
        
        <h3>3.3. Análisis de Hiperparámetros</h3>
        
        <div class="container">
            <div class="chart-container">
                <h4>Impacto del Número de Capas (LSTM)</h4>
                <img src="impact_n_layers_f1.png" alt="Impacto del Número de Capas" width="100%">
            </div>
            
            <div class="chart-container">
                <h4>Impacto de la Tasa de Aprendizaje (LSTM)</h4>
                <img src="impact_learning_rate_f1.png" alt="Impacto de la Tasa de Aprendizaje" width="100%">
            </div>
            
            <div class="chart-container">
                <h4>Impacto del Número de Capas (Transformer)</h4>
                <img src="impact_n_layers_f1.png" alt="Impacto del Número de Capas (Transformer)" width="100%">
            </div>
            
            <div class="chart-container">
                <h4>Impacto del Número de Cabezas (Transformer)</h4>
                <img src="impact_n_heads_f1.png" alt="Impacto del Número de Cabezas" width="100%">
            </div>
        </div>
        
        <h2>4. Análisis Comparativo</h2>
        
        <h3>4.1. Mejor Modelo RNN/LSTM vs Transformer</h3>
    """
    
    # Comparación del mejor modelo RNN/LSTM vs Transformer
    html_content += f"""
        <p>
            El mejor modelo de la familia RNN/LSTM fue <strong>{best_rnn_lstm_model}</strong> con un F1-score de {all_metrics[best_rnn_lstm_model]['f1']:.4f}.
            En comparación, el modelo Transformer obtuvo un F1-score de {all_metrics['Transformer']['f1']:.4f}.
        </p>
        
        <div class="container">
            <div class="chart-container full-width">
                <h4>Comparación Final: {best_rnn_lstm_model} vs Transformer</h4>
                <img src="final_comparison_f1.png" alt="Comparación Final" width="100%">
            </div>
        </div>
    """
    
    # Conclusiones basadas en los resultados
    html_content += """
        <h3>4.2. Componentes Clave del Transformer</h3>
        <p>
            El modelo Transformer se distingue por varios componentes clave que contribuyen a su rendimiento:
        </p>
        <ul>
            <li><strong>Mecanismo de Autoatención:</strong> Permite al modelo atender a diferentes partes de la secuencia de entrada simultáneamente, facilitando la captura de dependencias a larga distancia.</li>
            <li><strong>Codificación Posicional:</strong> Proporciona información sobre la posición de cada token en la secuencia, compensando la falta de recurrencia.</li>
            <li><strong>Arquitectura Encoder-Decoder:</strong> Permite procesar la entrada y generar la salida de manera eficiente, con un flujo de información bien estructurado.</li>
            <li><strong>Multi-Head Attention:</strong> Permite al modelo atender a diferentes representaciones del espacio simultáneamente, capturando diferentes tipos de relaciones.</li>
        </ul>
        
        <h2>5. Conclusiones</h2>
        
        <div class="conclusion">
    """
    
    # Conclusiones basadas en los resultados
    if all_metrics['Transformer']['f1'] > all_metrics[best_rnn_lstm_model]['f1']:
        html_content += f"""
            <p>
                <strong>1. Comparación de Arquitecturas:</strong><br>
                El modelo Transformer superó al mejor modelo RNN/LSTM ({best_rnn_lstm_model}) en términos de F1-score,
                demostrando su capacidad superior para capturar relaciones complejas en los datos.
            </p>
        """
    else:
        html_content += f"""
            <p>
                <strong>1. Comparación de Arquitecturas:</strong><br>
                El mejor modelo RNN/LSTM ({best_rnn_lstm_model}) superó al Transformer en términos de F1-score,
                lo que sugiere que para este conjunto de datos específico, las arquitecturas recurrentes pueden ser más adecuadas.
            </p>
        """
    
    if all_metrics['Transformer']['bleu'] > all_metrics[best_rnn_lstm_model]['bleu']:
        html_content += f"""
            <p>
                En términos de BLEU score, el Transformer también mostró un mejor rendimiento, indicando su superioridad
                para tareas de generación de texto.
            </p>
        """
    else:
        html_content += f"""
            <p>
                En términos de BLEU score, el modelo {best_rnn_lstm_model} mostró un mejor rendimiento, lo que sugiere
                que puede ser más adecuado para ciertas tareas de generación de texto en este contexto.
            </p>
        """
    
    if relative_times['Transformer'] < relative_times[best_rnn_lstm_model]:
        html_content += f"""
            <p>
                En cuanto a eficiencia computacional, el Transformer fue más rápido en inferencia que el mejor modelo RNN/LSTM,
                lo que destaca otra ventaja de su arquitectura paralela.
            </p>
        """
    else:
        html_content += f"""
            <p>
                En cuanto a eficiencia computacional, el modelo {best_rnn_lstm_model} fue más rápido en inferencia que el Transformer,
                lo que puede ser una consideración importante para aplicaciones con restricciones de recursos.
            </p>
        """
    
    html_content += """
            <p>
                <strong>2. Impacto de Hiperparámetros:</strong><br>
                - Número de capas: Un mayor número de capas puede mejorar el rendimiento hasta cierto punto, pero también aumenta el riesgo de sobreajuste.<br>
                - Tasa de aprendizaje: Una tasa de aprendizaje adecuada es crucial para la convergencia del modelo.<br>
                - Número de cabezas de atención (Transformer): Más cabezas permiten capturar diferentes tipos de relaciones en los datos.
            </p>
            
            <p>
                <strong>3. Ventajas y Desventajas:</strong><br>
                - RNN/LSTM:
                <ul>
                    <li class="advantage">Ventajas: Más simples, menos parámetros, eficientes para secuencias cortas.</li>
                    <li class="disadvantage">Desventajas: Dificultad para capturar dependencias a largo plazo, procesamiento secuencial.</li>
                </ul>
                
                - Transformer:
                <ul>
                    <li class="advantage">Ventajas: Paralelización, mejor captura de dependencias a largo plazo, atención a diferentes partes de la secuencia.</li>
                    <li class="disadvantage">Desventajas: Mayor número de parámetros, requiere más datos para entrenar efectivamente.</li>
                </ul>
            </p>
        </div>
        
        <h2>6. Recomendaciones</h2>
        <p>
            Basado en los resultados de este estudio, se pueden hacer las siguientes recomendaciones:
        </p>
        <ul>
            <li>Para tareas de NLP con secuencias largas y dependencias a distancia, considerar el uso de Transformers.</li>
            <li>Para aplicaciones con recursos limitados o conjuntos de datos pequeños, los modelos LSTM/GRU pueden ser más adecuados.</li>
            <li>La selección del modelo debe considerar no solo la precisión, sino también la eficiencia computacional según los requisitos específicos.</li>
            <li>Es recomendable realizar un ajuste cuidadoso de hiperparámetros, especialmente el número de capas y la tasa de aprendizaje.</li>
            <li>Para aplicaciones en tiempo real, considerar el equilibrio entre precisión y tiempo de inferencia.</li>
        </ul>
        
        <div class="footer">
            <p>Informe generado automáticamente - Análisis de Modelos RNN/LSTM y Transformer para NLP</p>
            <p>Fecha: """ + time.strftime("%d/%m/%Y") + """</p>
        </div>
    </body>
    </html>
    """
    
    # Guardar el informe HTML
    with open('informe_final.html', 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print("Informe final generado como 'informe_final.html'")


In [ ]:
# Generar informe final
generate_final_report(all_metrics, relative_times, model_names, best_rnn_lstm_model)

# Función para generar respuestas con el modelo
def generate_response(model, text_processor, input_text, device, temperature=0.8, beam_size=3):
    """
    Genera una respuesta utilizando el modelo entrenado
    """
    model.eval()
    
    # Convertir texto de entrada a índices
    input_indices = text_processor.text_to_indices(input_text, add_special_tokens=True)
    input_tensor = torch.tensor([input_indices], dtype=torch.long).to(device)
    
    # Para beam search
    if beam_size > 1:
        return beam_search_decode(model, input_tensor, text_processor, beam_size, max_length=50)
    
    # Para generación greedy o con temperatura
    with torch.no_grad():
        # Inicializar con token SOS
        output_indices = [text_processor.word2idx['<SOS>']]
        
        # Generar tokens uno a uno
        for _ in range(50):  # Limitar a 50 tokens como máximo
            # Convertir secuencia actual a tensor
            output_tensor = torch.tensor([output_indices], dtype=torch.long).to(device)
            
            # Obtener predicción del modelo
            with torch.no_grad():
                predictions = model(output_tensor)
            
            # Obtener distribución de probabilidad para el último token
            next_token_logits = predictions[0, -1, :]
            
            # Aplicar temperatura si es necesario
            if temperature != 1.0:
                next_token_logits = next_token_logits / temperature
            
            # Convertir a probabilidades
            next_token_probs = F.softmax(next_token_logits, dim=0)
            
            # Muestrear de la distribución o tomar el argmax
            if temperature > 0:
                next_token = torch.multinomial(next_token_probs, 1).item()
            else:
                next_token = torch.argmax(next_token_probs).item()
            
            # Añadir token a la secuencia
            output_indices.append(next_token)
            
            # Detener si se genera EOS
            if next_token == text_processor.word2idx['<EOS>']:
                break
    
    # Convertir índices a texto
    response = text_processor.indices_to_text(output_indices)
    return response


In [ ]:
def beam_search_decode(model, input_tensor, text_processor, beam_size=3, max_length=50):
    """
    Implementa beam search para generar respuestas de mayor calidad
    """
    # Inicializar con token SOS
    sequences = [[text_processor.word2idx['<SOS>']], 0.0]
    
    # Expandir secuencias
    for _ in range(max_length):
        all_candidates = []
        
        # Expandir cada secuencia actual
        for seq, score in sequences:
            if seq[-1] == text_processor.word2idx['<EOS>']:
                # Si la secuencia ya terminó, mantenerla como está
                all_candidates.append([seq, score])
                continue
            
            # Preparar entrada para el modelo
            output_tensor = torch.tensor([seq], dtype=torch.long).to(input_tensor.device)
            
            # Obtener predicción del modelo
            with torch.no_grad():
                predictions = model(output_tensor)
            
            # Obtener distribución para el último token
            next_token_logits = predictions[0, -1, :]
            next_token_probs = F.softmax(next_token_logits, dim=0)
            
            # Obtener los top-k tokens
            topk_probs, topk_indices = torch.topk(next_token_probs, beam_size)
            
            # Crear nuevas secuencias candidatas
            for i in range(beam_size):
                next_token = topk_indices[i].item()
                next_score = score - torch.log(topk_probs[i]).item()  # Usar log-probabilidad negativa
                
                all_candidates.append([seq + [next_token], next_score])
        
        # Ordenar candidatos por puntuación
        all_candidates.sort(key=lambda x: x[1])
        
        # Seleccionar los mejores beam_size candidatos
        sequences = all_candidates[:beam_size]
        
        # Verificar si todas las secuencias han terminado
        if all(seq[-1] == text_processor.word2idx['<EOS>'] for seq, _ in sequences):
            break
    
    # Tomar la mejor secuencia
    best_seq = sequences[0][0]
    
    # Convertir a texto
    response = text_processor.indices_to_text(best_seq)
    return response


In [ ]:
# Función para visualizar la atención del Transformer
def visualize_attention(model, text_processor, input_text, device):
    """
    Visualiza los pesos de atención del modelo Transformer
    """
    if not isinstance(model, TransformerModel):
        print("Esta función solo es compatible con modelos Transformer")
        return
    
    model.eval()
    
    # Convertir texto de entrada a índices
    input_indices = text_processor.text_to_indices(input_text, add_special_tokens=True)
    input_tensor = torch.tensor([input_indices], dtype=torch.long).to(device)
    
    # Obtener tokens de entrada como texto
    input_tokens = [text_processor.idx2word.get(idx, '<UNK>') for idx in input_indices]
    
    # Registrar hooks para capturar la atención
    attention_maps = []
    
    def get_attention(module, input, output):
        attention_maps.append(output[1].detach().cpu())
    
    # Registrar hooks en las capas de atención
    hooks = []
    for name, module in model.named_modules():
        if "multihead_attn" in name:
            hook = module.register_forward_hook(get_attention)
            hooks.append(hook)
    
    # Forward pass
    with torch.no_grad():
        _ = model(input_tensor)
    
    # Eliminar hooks
    for hook in hooks:
        hook.remove()
    
    # Visualizar mapas de atención
    if attention_maps:
        n_layers = len(attention_maps)
        n_heads = attention_maps[0].size(1)
        
        fig, axes = plt.subplots(n_layers, n_heads, figsize=(n_heads*3, n_layers*3))
        
        if n_layers == 1:
            axes = [axes]
        
        for i, layer_attention in enumerate(attention_maps):
            for j in range(n_heads):
                ax = axes[i][j] if n_heads > 1 else axes[i]
                
                # Obtener mapa de atención para esta cabeza
                attn = layer_attention[0, j].numpy()
                
                # Crear heatmap
                im = ax.imshow(attn, cmap='viridis')
                
                # Configurar etiquetas
                ax.set_xticks(range(len(input_tokens)))
                ax.set_yticks(range(len(input_tokens)))
                ax.set_xticklabels(input_tokens, rotation=90)
                ax.set_yticklabels(input_tokens)
                
                ax.set_title(f"Layer {i+1}, Head {j+1}")
        
        plt.tight_layout()
        plt.savefig('attention_visualization.png')
        plt.close()
        
        print("Visualización de atención guardada como 'attention_visualization.png'")
    else:
        print("No se pudieron capturar mapas de atención")


In [ ]:
# Función para analizar errores comunes
def analyze_errors(model, dataloader, text_processor, device, num_examples=10):
    """
    Analiza los errores más comunes del modelo
    """
    print(f"\nAnalizando errores comunes del modelo...")
    model.eval()
    errors = []
    
    with torch.no_grad():
        for src, trg in dataloader:
            if len(errors) >= num_examples:
                break
                
            src, trg = src.to(device), trg.to(device)
            output = model(src)
            
            # Obtener predicciones
            predictions = torch.argmax(output, dim=2)
            
            # Analizar cada ejemplo en el batch
            for i in range(src.size(0)):
                if len(errors) >= num_examples:
                    break
                
                # Convertir a listas de tokens
                input_tokens = [text_processor.idx2word.get(idx.item(), '<UNK>') for idx in src[i] if idx.item() > 0]
                target_tokens = [text_processor.idx2word.get(idx.item(), '<UNK>') for idx in trg[i] if idx.item() > 0]
                pred_tokens = [text_processor.idx2word.get(idx.item(), '<UNK>') for idx in predictions[i] if idx.item() > 0]
                
                # Calcular similitud
                target_text = ' '.join(target_tokens)
                pred_text = ' '.join(pred_tokens)
                
                # Si hay diferencia, registrar como error
                if target_text != pred_text:
                    # Encontrar tokens incorrectos
                    incorrect_tokens = []
                    for j, (t, p) in enumerate(zip(target_tokens, pred_tokens)):
                        if t != p:
                            incorrect_tokens.append((j, t, p))
                    
                    errors.append({
                        'input': ' '.join(input_tokens),
                        'target': target_text,
                        'prediction': pred_text,
                        'incorrect_tokens': incorrect_tokens
                    })
    
    # Mostrar errores
    print("\nAnálisis de errores comunes:")
    for i, error in enumerate(errors):
        print(f"\nError {i+1}:")
        print(f"Entrada: {error['input']}")
        print(f"Objetivo: {error['target']}")
        print(f"Predicción: {error['prediction']}")
        print("Tokens incorrectos:")
        for pos, target, pred in error['incorrect_tokens']:
            print(f"  Posición {pos}: '{target}' -> '{pred}'")
    
    # Analizar patrones de error
    error_patterns = {}
    for error in errors:
        for _, target, pred in error['incorrect_tokens']:
            pattern = f"{target} -> {pred}"
            if pattern in error_patterns:
                error_patterns[pattern] += 1
            else:
                error_patterns[pattern] = 1
    
    # Mostrar patrones más comunes
    print("\nPatrones de error más comunes:")
    sorted_patterns = sorted(error_patterns.items(), key=lambda x: x[1], reverse=True)
    for pattern, count in sorted_patterns[:5]:
        print(f"  {pattern}: {count} ocurrencias")
    
    # Crear visualización
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['Error', 'Entrada', 'Objetivo', 'Predicción', 'Tokens Incorrectos'],
            fill_color='paleturquoise',
            align='left'
        ),
        cells=dict(
            values=[
                list(range(1, len(errors) + 1)),
                [error['input'] for error in errors],
                [error['target'] for error in errors],
                [error['prediction'] for error in errors],
                [', '.join([f"'{t}'->{p}" for _, t, p in error['incorrect_tokens']]) for error in errors]
            ],
            fill_color='lavender',
            align='left'
        )
    )])
    
    fig.update_layout(
        title="Análisis de Errores",
        height=125 * len(errors)
    )
    
    display(fig)
    fig.write_html('error_analysis.html')
    print("Análisis de errores guardado como 'error_analysis.html'")
    
    return errors

In [ ]:
# Ejecutar análisis de errores para el mejor modelo
print("\n¿Deseas realizar un análisis de errores del mejor modelo? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    # Usar el mejor modelo según las métricas
    best_model_name = max(['RNN', 'LSTM', 'GRU', 'Transformer'], 
                          key=lambda x: all_metrics[x]['f1'])
    
    print(f"Realizando análisis de errores del modelo {best_model_name}...")
    
    if best_model_name == 'RNN':
        best_model = rnn_model
    elif best_model_name == 'LSTM':
        best_model = lstm_model
    elif best_model_name == 'GRU':
        best_model = gru_model
    else:
        best_model = transformer_model
    
    # Ejecutar análisis de errores
    error_analysis = analyze_errors(best_model, test_loader, text_processor, device)

# Si el mejor modelo es un Transformer, visualizar la atención
if best_model_name == 'Transformer':
    print("\n¿Deseas visualizar los mapas de atención del Transformer? (s/n)")
    choice = input()
    if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
        print("Ingresa un texto para visualizar la atención:")
        input_text = input()
        visualize_attention(transformer_model, text_processor, input_text, device)

In [ ]:
# Función para evaluar la robustez del modelo
def evaluate_robustness(model, text_processor, device, num_examples=5):
    """
    Evalúa la robustez del modelo frente a perturbaciones en la entrada
    """
    print("\nEvaluando robustez del modelo frente a perturbaciones...")
    
    # Obtener algunos ejemplos del conjunto de prueba
    examples = []
    for src, trg in test_loader:
        if len(examples) >= num_examples:
            break
        
        for i in range(min(src.size(0), num_examples - len(examples))):
            input_text = text_processor.indices_to_text(src[i].numpy())
            target_text = text_processor.indices_to_text(trg[i].numpy())
            
            if len(input_text.split()) > 5:  # Asegurar que el texto tenga suficientes palabras
                examples.append({
                    'input': input_text,
                    'target': target_text
                })
    
    # Tipos de perturbaciones
    perturbations = [
        ('original', lambda x: x),
        ('eliminar_palabra', lambda x: ' '.join(x.split()[1:] if len(x.split()) > 1 else x.split())),
        ('cambiar_orden', lambda x: ' '.join(x.split()[::-1]) if len(x.split()) > 1 else x),
        ('duplicar_palabra', lambda x: x + ' ' + x.split()[0] if len(x.split()) > 0 else x),
        ('añadir_ruido', lambda x: x + ' xyz123')
    ]
    
    # Evaluar cada ejemplo con diferentes perturbaciones
    results = []
    
    for example in examples:
        example_results = {'original': example}
        
        for name, perturb_func in perturbations:
            if name == 'original':
                perturbed_input = example['input']
            else:
                perturbed_input = perturb_func(example['input'])
            
            # Generar respuesta con el modelo
            model_output = generate_response(model, text_processor, perturbed_input, device)
            
            example_results[name] = {
                'input': perturbed_input,
                'output': model_output
            }
        
        results.append(example_results)
    
    # Mostrar resultados
    print("\nResultados de la evaluación de robustez:")
    
    for i, result in enumerate(results):
        print(f"\nEjemplo {i+1}:")
        print(f"Original - Entrada: {result['original']['input']}")
        print(f"Original - Objetivo: {result['original']['target']}")
        
        for name in [p[0] for p in perturbations]:
            if name != 'original':
                print(f"\n{name.capitalize()} - Entrada: {result[name]['input']}")
                print(f"{name.capitalize()} - Salida: {result[name]['output']}")
    
    # Crear visualización
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['Ejemplo', 'Tipo', 'Entrada', 'Salida'],
            fill_color='paleturquoise',
            align='left'
        ),
        cells=dict(
            values=[
                [f"Ejemplo {i+1}" for i in range(len(results)) for _ in range(len(perturbations))],
                [p[0].capitalize() for _ in range(len(results)) for p in perturbations],
                [result[p[0]]['input'] for result in results for p in perturbations],
                [result[p[0]].get('output', result[p[0]].get('target', '')) for result in results for p in perturbations]
            ],
            fill_color='lavender',
            align='left'
        )
    )])
    
    fig.update_layout(
        title="Evaluación de Robustez",
        height=125 * len(results) * len(perturbations)
    )
    
    display(fig)
    fig.write_html('robustness_evaluation.html')
    print("Evaluación de robustez guardada como 'robustness_evaluation.html'")
    
    return results


In [ ]:
# Ejecutar evaluación de robustez para el mejor modelo
print("\n¿Deseas realizar una evaluación de robustez del mejor modelo? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    # Usar el mejor modelo según las métricas
    best_model_name = max(['RNN', 'LSTM', 'GRU', 'Transformer'], 
                          key=lambda x: all_metrics[x]['f1'])
    
    print(f"Realizando evaluación de robustez del modelo {best_model_name}...")
    
    if best_model_name == 'RNN':
        best_model = rnn_model
    elif best_model_name == 'LSTM':
        best_model = lstm_model
    elif best_model_name == 'GRU':
        best_model = gru_model
    else:
        best_model = transformer_model
    
    # Ejecutar evaluación de robustez
    robustness_results = evaluate_robustness(best_model, text_processor, device)

In [ ]:
# Función para comparar la eficiencia de memoria
def compare_memory_usage(models, model_names, device):
    """
    Compara el uso de memoria de diferentes modelos
    """
    print("\nComparando uso de memoria de los modelos...")
    
    memory_usage = {}
    
    for model, name in zip(models, model_names):
        # Mover modelo a CPU para medición precisa
        model = model.to('cpu')
        
        # Calcular número de parámetros
        num_params = sum(p.numel() for p in model.parameters())
        
        # Estimar uso de memoria (en MB)
        memory_estimate = num_params * 4 / (1024 * 1024)  # 4 bytes por parámetro (float32)
        
        memory_usage[name] = {
            'params': num_params,
            'memory_mb': memory_estimate
        }
        
        # Devolver modelo al dispositivo original
        model = model.to(device)
    
    # Mostrar resultados
    print("\nUso de memoria por modelo:")
    for name, usage in memory_usage.items():
        print(f"{name}: {usage['params']:,} parámetros, {usage['memory_mb']:.2f} MB")
    
    # Crear visualización
    fig = go.Figure()
    
    # Gráfico de barras para parámetros
    fig.add_trace(go.Bar(
        x=list(memory_usage.keys()),
        y=[usage['params'] for usage in memory_usage.values()],
        name='Número de Parámetros',
        text=[f"{usage['params']:,}" for usage in memory_usage.values()],
        textposition='auto'
    ))
    
    fig.update_layout(
        title='Comparación de Número de Parámetros',
        xaxis_title='Modelo',
        yaxis_title='Número de Parámetros',
        template='plotly_white'
    )
    
    display(fig)
    fig.write_html('memory_comparison.html')
    
    # Gráfico de barras para memoria
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        x=list(memory_usage.keys()),
        y=[usage['memory_mb'] for usage in memory_usage.values()],
        name='Memoria (MB)',
        text=[f"{usage['memory_mb']:.2f} MB" for usage in memory_usage.values()],
        textposition='auto'
    ))
    
    fig2.update_layout(
        title='Comparación de Uso de Memoria',
        xaxis_title='Modelo',
        yaxis_title='Memoria (MB)',
        template='plotly_white'
    )
    
    display(fig2)
    fig2.write_html('memory_comparison_mb.html')
    
    print("Comparación de memoria guardada como 'memory_comparison.html' y 'memory_comparison_mb.html'")
    
    return memory_usage

# Ejecutar comparación de memoria
print("\n¿Deseas comparar el uso de memoria de los modelos? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    memory_usage = compare_memory_usage(
        [rnn_model, lstm_model, gru_model, transformer_model],
        ['RNN', 'LSTM', 'GRU', 'Transformer'],
        device
    )


In [ ]:
# Función para generar un resumen ejecutivo
def generate_executive_summary(all_metrics, relative_times, memory_usage, best_model_name):
    """
    Genera un resumen ejecutivo con los principales hallazgos
    """
    print("\nGenerando resumen ejecutivo...")
    
    summary = """
# Resumen Ejecutivo: Comparación de Modelos RNN/LSTM y Transformer para NLP

## Objetivo
Este estudio comparó diferentes arquitecturas de redes neuronales para procesamiento de lenguaje natural (NLP): RNN simple, LSTM, GRU y Transformer, evaluando su rendimiento, eficiencia y características.

## Metodología
Se entrenaron y evaluaron los modelos utilizando métricas estándar de NLP como precisión, F1-score, BLEU y ROUGE, además de medir tiempos de inferencia y uso de memoria.

## Principales Hallazgos
"""
    
    # Mejor modelo según F1-score
    best_f1_model = max(model_names, key=lambda x: all_metrics[x]['f1'])
    summary += f"1. **Rendimiento**: El modelo {best_f1_model} obtuvo el mejor F1-score ({all_metrics[best_f1_model]['f1']:.4f}), "
    
    # Mejor modelo según BLEU
    best_bleu_model = max(model_names, key=lambda x: all_metrics[x]['bleu'])
    if best_bleu_model == best_f1_model:
        summary += f"y también el mejor BLEU score ({all_metrics[best_bleu_model]['bleu']:.4f}).\n"
    else:
        summary += f"mientras que {best_bleu_model} obtuvo el mejor BLEU score ({all_metrics[best_bleu_model]['bleu']:.4f}).\n"
    
    # Modelo más rápido
    fastest_model = min(model_names, key=lambda x: relative_times[x])
    summary += f"2. **Eficiencia**: {fastest_model} fue el modelo más rápido en inferencia, "
    
    # Modelo con menos memoria
    if memory_usage:
        smallest_model = min(model_names, key=lambda x: memory_usage[x]['memory_mb'])
        summary += f"y {smallest_model} utilizó la menor cantidad de memoria ({memory_usage[smallest_model]['memory_mb']:.2f} MB).\n"
    else:
        summary += "siendo 1.0x más rápido que el segundo mejor.\n"
    
    # Comparación RNN/LSTM vs Transformer
    summary += "3. **Comparación RNN/LSTM vs Transformer**:\n"
    
    if all_metrics['Transformer']['f1'] > all_metrics[best_rnn_lstm_model]['f1']:
        summary += f"   - El Transformer superó al mejor modelo RNN/LSTM en F1-score ({all_metrics['Transformer']['f1']:.4f} vs {all_metrics[best_rnn_lstm_model]['f1']:.4f}).\n"
    else:
        summary += f"   - El mejor modelo RNN/LSTM ({best_rnn_lstm_model}) superó al Transformer en F1-score ({all_metrics[best_rnn_lstm_model]['f1']:.4f} vs {all_metrics['Transformer']['f1']:.4f}).\n"
    
    if relative_times['Transformer'] < relative_times[best_rnn_lstm_model]:
        summary += f"   - El Transformer fue más rápido en inferencia que el mejor modelo RNN/LSTM ({relative_times['Transformer']:.2f}x vs {relative_times[best_rnn_lstm_model]:.2f}x).\n"
    else:
        summary += f"   - El mejor modelo RNN/LSTM fue más rápido en inferencia que el Transformer ({relative_times[best_rnn_lstm_model]:.2f}x vs {relative_times['Transformer']:.2f}x).\n"
    
    if memory_usage:
        if memory_usage['Transformer']['memory_mb'] < memory_usage[best_rnn_lstm_model]['memory_mb']:
            summary += f"   - El Transformer utilizó menos memoria que el mejor modelo RNN/LSTM ({memory_usage['Transformer']['memory_mb']:.2f} MB vs {memory_usage[best_rnn_lstm_model]['memory_mb']:.2f} MB).\n"
        else:
            summary += f"   - El mejor modelo RNN/LSTM utilizó menos memoria que el Transformer ({memory_usage[best_rnn_lstm_model]['memory_mb']:.2f} MB vs {memory_usage['Transformer']['memory_mb']:.2f} MB).\n"
    
    # Conclusiones
    summary += """
## Conclusiones
1. **Selección de modelo**: La elección entre arquitecturas RNN/LSTM y Transformer debe considerar el equilibrio entre rendimiento, velocidad y uso de recursos según los requisitos específicos de la aplicación.

2. **Ventajas y desventajas**:
   - **RNN/LSTM**: Más simples, menos parámetros, eficientes para secuencias cortas. Sin embargo, tienen dificultad para capturar dependencias a largo plazo.
   - **Transformer**: Mejor paralelización y captura de dependencias a largo plazo, pero requieren más datos para entrenar efectivamente.

3. **Recomendaciones**:
   - Para tareas con secuencias largas y dependencias a distancia: considerar Transformers.
   - Para aplicaciones con recursos limitados o conjuntos de datos pequeños: considerar LSTM/GRU.
   - Realizar un ajuste cuidadoso de hiperparámetros, especialmente el número de capas y la tasa de aprendizaje.
"""
    
    # Guardar resumen
    with open('resumen_ejecutivo.md', 'w', encoding='utf-8') as f:
        f.write(summary)
    
    print("Resumen ejecutivo guardado como 'resumen_ejecutivo.md'")
    
    return summary


In [ ]:
# Generar resumen ejecutivo
print("\n¿Deseas generar un resumen ejecutivo de los resultados? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    memory_usage_data = memory_usage if 'memory_usage' in locals() else None
    executive_summary = generate_executive_summary(all_metrics, relative_times, memory_usage_data, best_model_name)
    print("\nResumen Ejecutivo:")
    print(executive_summary)

In [ ]:
# Función para crear una interfaz de chat simple
def run_chat_interface(model, text_processor, device, max_turns=5):
    """
    Crea una interfaz de chat simple para interactuar con el modelo
    """
    print("\n===== MINI CHAT CON EL MODELO =====")
    print("(Escribe 'salir' para terminar)")
    
    chat_history = []
    
    for turn in range(max_turns):
        # Obtener entrada del usuario
        user_input = input("\nTú: ")
        
        if user_input.lower() in ['salir', 'exit', 'quit']:
            break
        
        # Añadir a historial
        chat_history.append(f"Usuario: {user_input}")
        
        # Generar respuesta
        if len(chat_history) > 1:
            # Usar todo el historial como contexto
            context = " ".join(chat_history)
        else:
            context = user_input
        
        # Generar respuesta con beam search
        model_response = generate_response(model, text_processor, context, device, beam_size=3)
        
        # Mostrar respuesta
        print(f"Modelo: {model_response}")
        
        # Añadir a historial
        chat_history.append(f"Modelo: {model_response}")
    
    print("\nFin del chat.")
    return chat_history


In [ ]:
# Ejecutar interfaz de chat con el mejor modelo
print("\n¿Deseas probar el mejor modelo en un mini chat? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    # Usar el mejor modelo según las métricas
    best_model_name = max(['RNN', 'LSTM', 'GRU', 'Transformer'], 
                          key=lambda x: all_metrics[x]['f1'])
    
    print(f"Iniciando chat con el modelo {best_model_name}...")
    
    if best_model_name == 'RNN':
        chat_model = rnn_model
    elif best_model_name == 'LSTM':
        chat_model = lstm_model
    elif best_model_name == 'GRU':
        chat_model = gru_model
    else:
        chat_model = transformer_model
    
    # Ejecutar interfaz de chat
    chat_history = run_chat_interface(chat_model, text_processor, device)

In [ ]:
# Función para guardar los modelos entrenados
def save_models(models_dict, save_dir='modelos_entrenados'):
    """
    Guarda los modelos entrenados para uso futuro
    """
    print(f"\nGuardando modelos entrenados en '{save_dir}'...")
    
    # Crear directorio si no existe
    os.makedirs(save_dir, exist_ok=True)
    
    # Guardar cada modelo
    for name, model in models_dict.items():
        model_path = os.path.join(save_dir, f"{name}_model.pt")
        torch.save(model.state_dict(), model_path)
        print(f"Modelo {name} guardado en {model_path}")
    
    # Guardar el procesador de texto
    text_processor_path = os.path.join(save_dir, "text_processor.pkl")
    with open(text_processor_path, 'wb') as f:
        import pickle
        pickle.dump(text_processor, f)
    
    print(f"Procesador de texto guardado en {text_processor_path}")
    
    # Guardar configuración
    config = {
        'INPUT_DIM': INPUT_DIM,
        'OUTPUT_DIM': OUTPUT_DIM,
        'EMB_DIM': EMB_DIM,
        'HIDDEN_DIM': HIDDEN_DIM,
        'N_LAYERS': N_LAYERS,
        'N_HEADS': N_HEADS,
        'DROPOUT': DROPOUT,
        'LEARNING_RATE': LEARNING_RATE,
        'N_EPOCHS': N_EPOCHS,
        'metrics': {name: {k: float(v) for k, v in metrics.items()} for name, metrics in all_metrics.items()},
        'relative_times': {k: float(v) for k, v in relative_times.items()}
    }
    
    config_path = os.path.join(save_dir, "config.json")
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)
    
    print(f"Configuración guardada en {config_path}")
    
    return save_dir


In [ ]:
# Preguntar si se desean guardar los modelos
print("\n¿Deseas guardar los modelos entrenados? (s/n)")
choice = input()
if choice.lower() in ['s', 'si', 'sí', 'y', 'yes']:
    models_dict = {
        'RNN': rnn_model,
        'LSTM': lstm_model,
        'GRU': gru_model,
        'Transformer': transformer_model
    }
    
    save_dir = save_models(models_dict)
    print(f"Todos los modelos guardados en '{save_dir}'")